In [1]:
#Example pub med ids
pmids = [
    "30530648", "31820734", "31018141", "38539015", "33763704", "32572264", "31002671", 
    "33309739", "21057496", "27716510", "34059805", "34941412", "33879573", "35440059", 
    "33879573", "29462153", "29794063", "25939354", "30322904", "36879017", "35419551", 
    "31501549", "23042784", "22219169", "20670891", "20385583", "19723310", "20602769", 
    "26566685", "38030723", "30498128", "20485568", "21613409", "31076851", "37169753", 
    "39800688", "36510023", "38177678", "36539615", "37871105", "35235788", "27799057", 
    "30820472", "31666070", "34686734", "22384383", "34033742", "33589615", "31792364", 
    "29576475", "39762647", "32084358", "38977847", "25493933", "31125107", "26749252", 
    "39587714", "30333487", "33743111", "35172154", "32025611", "31136284", "26740022", 
    "37989753", "39838364", "39367016", "36650381", "35511946", "38641753", "38287646", 
    "36840360", "36544018", "36840360", "39637179", "35767948", "31801092", "38909241", 
    "36544018", "32384151", "26280576", "38379415", "29550329", "19211887", "36803569", 
    "30320226", "35920937", "37958987", "25340342", "37277533", "24223949"
]


In [2]:
import requests
from xml.etree import ElementTree

def fetch_geo_ids_bulk(pubmed_ids):
    #Using elink to get all the connected links
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"
    
    joined_ids = ",".join(pubmed_ids)
    full_url = f"{base_url}?dbfrom=pubmed&db=gds&id={joined_ids}&retmode=xml"

    response = requests.get(full_url)

    geo_ids = []

    if response.status_code == 200:
        root = ElementTree.fromstring(response.content)

        for link in root.findall(".//LinkSetDb[DbTo='gds']/Link/Id"):
            geo_ids.append(link.text)
    else:
        print(f"Error: {response.status_code}")

    return geo_ids

In [3]:
geo_ids = fetch_geo_ids_bulk(pmids)

In [4]:
geo_ids[:5]

['200272793', '200268524', '200263013', '200261729', '200254970']

In [5]:
import re
def get_accession(text):
    match = re.search(r"Series\s+Accession:\s+(GSE\d+)", text)
    return match.group(1) if match else None

In [6]:
get_accession('''1. An atlas of small non-coding RNAs in Human Preimplantation Development [RNA-seq]
(Submitter supplied) Our current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. more...
Organism:	Homo sapiens
Type:		Expression profiling by high throughput sequencing
Platform: GPL24676 103 Samples
FTP download: GEO (TSV) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE249nnn/GSE249493/
Series		Accession: GSE249493	ID: 200249493''')

'GSE249493'

In [7]:
from bs4 import BeautifulSoup
def extract_data(page_html):
    soup = BeautifulSoup(page_html, 'html.parser')

    data = ""
    fields = {
        "title": "Title",
        "summary": "Summary",
        "organism": "Organism",
        "experiment": "Experiment type",
        "overall_design": "Overall design"
    }

    for key, label in fields.items():
        tag = soup.find(string=label)
        if tag:
            value = tag.find_next("td").text.strip()
            data += value
            data += '\n'

    return data

In [9]:
sample_page_html = '''
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<HTML>
  <HEAD>
    
    <style type="text/css">
      a { text-decoration: none; }
	.bordergc {background-color: #6699CC;}
	.bordergd {background-color: #B6C7E5;}
	.borderge {background-color: #EEF3FB;}
	.bordergf {background-color: #FFFFFF;}
	.bordergg {background-color: #CCCCCC;}
      .small8b { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#6633cc;
              }
      .small8db { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#4411aa;
              }

    </style>
    <META http-equiv="Content-Type"
      content="text/html; charset=UTF-8">
    <META name="keywords"
      CONTENT="NCBI GEO Gene Expression Omnibus microarray oligonucleotide array SAGE">
    <META name="description"
      content="NCBI's Gene Expression Omnibus (GEO) is a public archive and resource for gene expression data.">

<meta name="ncbi_app" content="geo">
<meta name="ncbi_pdid" content="full">
<meta name="ncbi_phid" content="075178DA7CD92BD10000000000000001">
<meta name="ncbi_sessionid" content="075178DA7CD92BD1_0000SID">

    <TITLE>
    GEO Accession viewer
    </TITLE>
    <link rel="stylesheet"
      href="/corehtml/ncbi.css">
    <!-- GEO_SCRIPT -->

<SCRIPT LANGUAGE="JavaScript1.2"
SRC="/coreweb/javascript/imagemouseover.js"></SCRIPT>

<SCRIPT LANGUAGE="JavaScript1.2"
SRC="/coreweb/javascript/show_message.js"></SCRIPT>

<script type="text/javascript" src="/corehtml/jsutils/utils.1.js"></script>

<script type="text/javascript" src="/corehtml/jsutils/remote_data_provider.1.js"></script>

<SCRIPT LANGUAGE="JavaScript1.2"
SRC="/geo/js/help_def_messages.js"></SCRIPT>

<script type="text/javascript">
    window.onload = function () {
        jQuery.getScript("/core/alerts/alerts.js", function () {
            galert(['#galerts_table','body > *:nth-child(1)'])
        });
    }
</script>



<LINK  rel = STYLESHEET href = "../info/geo_style.css" Type  = "text/css" >
<link rel="stylesheet" type="text/css" href="acc.css" />
  <script language="Javascript">

  function OnFormFieldChange()
  {
    var view = document.getElementById("view");

    if(document.getElementById("ViewOptions").form.value == 'html')
    {
        view.remove(3);
        view.remove(2);
    }
    else
    {
        var NewOption = document.createElement("OPTION");

        NewOption.text = "Full";
        NewOption.value = "full";

        try
        {
            view.add(NewOption, null);
        }
        catch(ex)
        {
            view.add(NewOption);
        }

        NewOption = document.createElement("OPTION");

        NewOption.text = "Data";
        NewOption.value = "data";

        try
        {
            view.add(NewOption, null);
        }
        catch(ex)
        {
            view.add(NewOption);
        }
    }
  }

  function SubmitViewOptionsForm()
  {
	var form = document.forms.ViewOptions;
    if(form.form.value == 'html')
    {
		form.form.setAttribute('disabled','disabled');
		if (form.view.value == 'quick') {
			form.view.setAttribute('disabled','disabled');
		}
		if (form.targ.value == 'self') {
			form.targ.setAttribute('disabled','disabled');
		}
        var token = document.getElementById("token_input");
        if (token) {
            form.token.value = token.value;
        } else {
            form.token.setAttribute('disabled','disabled');
        }
        form.submit();
    }
    else
    {
        window.open("acc.cgi?acc=" + form.acc.value + "&targ=" + form.targ.value +
                  "&form=" + form.form.value + "&view=" + form.view.value, "_self");
    }

    return false;
  }
  
  function ViewOptionsFormKeyDown(event)
  {
	if (event == undefined)
	{    
		event = window.event;
	}
	if (event.keyCode == 13)
	{
		SubmitViewOptionsForm();
		return false;
	}
  };

  function OpenFTP(url)
  {
    window.open(url.replace('ftp://', 'https://'), '_blank');
  }

  function OpenLink(url, where)
  {
    window.open(url, where);
  }

  utils.addEvent(window, "load", OnFormFieldChange)
  </script>

</head>
<body background="/coreweb/template1/pix/bg_main3.gif" topmargin="20" marginheight="20">


<script type="text/javascript" src="/core/jig/1.15.10/js/jig.min.js"></script>
<script type="text/javascript" src="/corehtml/pmc/granthub/v1/granthubsearch.min.js"></script>
<script type="text/javascript" src="/geo/js/dd_menu.js"></script>
	<table width="740" border="0" cellspacing="0" cellpadding="0" align="center" >
			<tr>
				<td>
					<table width="100%" border="0" cellspacing="0" cellpadding="0" align="center">
						<tr>
							<td><a href="/"><img src="/geo/img/ncbi_logo.gif" alt="NCBI Logo" width="145" height="66" border="0"></a></td>
							<td width="100%" align="center" valign="middle" nowrap background="/coreweb/template1/pix/top_bg_white.gif"><img src="/coreweb/template1/pix/pixel.gif" width="550" height="1" alt="" border="0"><br>
								<a href="/geo/"><img src="/geo/img/geo_main.gif" alt="GEO Logo" border="0"></a>
							</td>
							<td align="right" background="/coreweb/template1/pix/top_bg_white.gif"><img src="/coreweb/template1/pix/top_right.gif" alt="" width="5" height="66" border="0"></td>
						</tr>
					</table>
					<table width="100%" border="0" cellspacing="0" cellpadding="0" align="center">
						<tr>
							<td><img src="/coreweb/template1/pix/top2_left.gif" width="601" height="2" alt="" border="0"></td>
							<td width="100%" background="/coreweb/template1/pix/top2_mid_bg.gif"><img src="/coreweb/template1/pix/pixel.gif" width="1" height="1" alt="" border="0"></td>
							<td align="right"><img src="/coreweb/template1/pix/top2_right.gif" alt="" width="14" height="2" border="0"></td>
						</tr>
					</table>
                    <table width="100%" border="0" cellspacing="0" cellpadding="0" align="center" id="galerts_table"/>
					<table width="100%" border="0" cellspacing="0" cellpadding="0" align="center">
						<tr>
							<td><img src="/coreweb/template1/pix/top3_ulm_no_a.gif" width="145" height="16" alt="" border="0" usemap="#unlmenu" name="unl_menu_pix"></td>
							<td background="/coreweb/template1/pix/top3_mainmenu_mid_bg.gif"><img src="/coreweb/template1/pix/top3_mainmenu_left.gif" width="3" height="16" alt="" border="0"></td>
							<td width="100%" valign="middle" nowrap background="/coreweb/template1/pix/top3_mainmenu_mid_bg.gif">

					<!-- GEO Navigation -->
			<ul id="geo_nav_bar">
				<li><a href="#">GEO Publications</a>
					<ul class="sublist">
						<li><a href="/geo/info/GEOHandoutFinal.pdf">Handout</a></li>
                        <li><a href="/pmc/articles/PMC10767856/">NAR 2024 (latest)</a></li>
						<li><a href="/pmc/articles/PMC99122/">NAR 2002 (original)</a></li>
						<li><a href="/pmc/?term=10767856,4944384,3531084,3341798,3013736,2686538,2270403,1669752,1619900,1619899,539976,99122">All publications</a></li>
					</ul>
				</li>
				<li><a href="/geo/info/faq.html">FAQ</a></li>
				<li><a href="/geo/info/MIAME.html" title="Minimum Information About a Microarray Experiment">MIAME</a></li>
				<li><a href="mailto:geo@ncbi.nlm.nih.gov">Email GEO</a></li>
			</ul>
			<!-- END GEO Navigation -->

                    </td>
                    <td background="/coreweb/template1/pix/top3_mainmenu_mid_bg.gif" align="right"><img src="/coreweb/template1/pix/top3_mainmenu_right.gif" width="5" height="16" alt="" border="0"></td>
                </tr>
            </table>
            
            <table width="100%" border="0" cellspacing="0" cellpadding="0" align="center">
                <tr>
                    <td><img src="/coreweb/template1/pix/top4_ulm_left.gif" width="145" height="4" alt="" border="0"></td>
                    <td width="100%" background="/coreweb/template1/pix/top4_mid_bg.gif"><img src="/coreweb/template1/pix/pixel.gif" width="1" height="1" alt="" border="0"></td>
                    <td align="right" background="/coreweb/template1/pix/top4_mid_bg.gif"><img src="/coreweb/template1/pix/top4_ulm_right.gif" width="5" height="4" alt="" border="0"></td>
                </tr>
            </table>
    
            <table width="100%" border="0" cellspacing="0" cellpadding="0" align="center">
                <tr>
                    <td width=1 background="/coreweb/template1/pix/main_left_bg.gif"><img src="/coreweb/template1/pix/main_left_bg.gif" alt="" width="4" height="3" border="0"></td>
                    <td width="10000" bgcolor="#F0F8FF">
                        <table cellpadding="0" cellspacing="0" width="100%"><tr><td><font class="Top_Navigation_text" color="#2F6E87" face="Verdana" size="+1">&nbsp;&nbsp;&nbsp;<a href="/">NCBI</a> &gt; <a href="/geo"><font color="">GEO</font></a> &gt; <a href="acc.cgi"><b>Accession Display</b></a><a href="javascript:RPopUpWindow_Set(geologinbar_location,260,120,'','','#E1EAE6','','#538AA9','MessageBox2');" onmouseout="RPopUpWindow_Stop()"><img alt="Help" height="11" src="/coreweb/images/long_help4.gif" style="border: none" width="19"></a></font></td>
<td align="right">Not logged in | <a href="/geo/submitter?ix=1NgRuQrLLZY2T5prqL8__-4iZbcPk5lG-995p1zcKkacFA14qHlaWbLdR6DeHUaPe5ihDsqxsuO_1VBZdgvkG">Login</a><a href="javascript:RPopUpWindow_Set(geologinbar_login,260,200,'','','#E1EAE6','','#538AA9','MessageBox2');" onmouseout="RPopUpWindow_Stop()"><img alt="Help" height="11" src="/coreweb/images/long_help4.gif" style="border: none" width="19"></a></td>
</tr></table>
                    </td>
                    <td width=1 background="/coreweb/template1/pix/main_right_bg.gif"><img src="/coreweb/template1/pix/main_right_bg.gif" width="4" height="3" alt="" border="0"></td>
                </tr>
                <tr>
                    <td background="/coreweb/template1/pix/main_left_bg.gif"><img src="/coreweb/template1/pix/main_left_bg.gif" width="4" height="1" alt="" border="0"></td>
                    <td width="10000" bgcolor="#E0EEEE"><img src="/coreweb/template1/pix/pixel.gif" width="1" height="1" alt="" border="0"></td>
                    <td align="right" background="/coreweb/template1/pix/main_right_bg.gif"><img src="/coreweb/template1/pix/main_right_bg.gif" alt="" width="4" height="1" border="0"></td>
                </tr>

                <tr>
                    <td background="/coreweb/template1/pix/main_left_bg.gif"><img src="/coreweb/template1/pix/main_left_bg.gif" width="4" height="3" alt="" border="0"></td>
                    <td width="100%" bgcolor="White">
                        <table width="98%" border="0" align="center">
                            <tr>
                                <td>
                                    <table border="0" cellspacing="0" cellpadding="0" align="right" width="100%">
                                        <tr>
                                            <td>

 <script type="text/javascript" src="acc.js"></script>
 <span id="msg_err" style="color:red"></span>
 <span id="msg_info" style="color:blue"></span>
<table cellpadding="0" cellspacing="0" style="border: 1px solid #C0F8FF"><tr><td><img alt=" " height="35" src="/coreweb/template1/pix/pixel.gif" width="1"></td>
<td bgcolor="#F0F8FF" width="100%"><font color="#0066CC" face="Arial" size="1"><div id="HelpMessage" style="font: 11px/11px arial, sans-serif"><strong>GEO help:</strong> Mouse over screen elements for information.</div></font></td>
</tr></table>
<form action="acc.cgi" enctype="application/x-www-form-urlencoded" id="ViewOptions" method="POST" name="ViewOptions" target="_self"><table border="0" cellpadding="0" cellspacing="0" width="100%"><tr><td></td>
<td bgcolor="#CCCCCC" nowrap valign="middle" width="100%"><table align="left" border="0" cellpadding="0" cellspacing="0"><tr><td nowrap><table border="0" cellpadding="0" cellspacing="0"><tr><td valign="middle"><input id="token" name="token" type="hidden" value=""><label for="scope">Scope: </label><select id="scope" name="targ" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_scope)" style="font-size: 10px"><option selected value="self">Self</option>
<option value="gpl">Platform</option>
<option value="gsm">Samples</option>
<option value="gse">Series</option>
<option value="all">Family</option>
</select>
&nbsp;&nbsp;<label for="form">Format: </label><select id="form" name="form" onchange="OnFormFieldChange()" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_format)" style="font-size: 10px"><option value="html">HTML</option>
<option value="text">SOFT</option>
<option value="xml">MINiML</option>
</select>
&nbsp;&nbsp;<label for="view">Amount: </label><select id="view" name="view" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_amount)" style="font-size: 10px"><option value="brief">Brief</option>
<option selected value="quick">Quick</option>
</select>
&nbsp;<label for="geo_acc">GEO accession: </label><input id="geo_acc" name="acc" onkeydown="ViewOptionsFormKeyDown(event)" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_acc)" style="font-size: 10px" type="text" value="GSE272793">&nbsp;&nbsp;</td>
<td valign="middle"><img alt="Go" border="0" onclick="SubmitViewOptionsForm()" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_go)" src="/geo/img/buttons/go_button.gif"></td>
</tr></table></td></tr></table></td>
</tr></table></form>
    <table><tr><td><table cellpadding="2" cellspacing="0" width="600"><tr bgcolor="#cccccc" valign="top"><td colspan="2"><table width="600"><tr><td><strong class="acc" id="GSE272793"><a href="/geo/query/acc.cgi?acc=GSE272793" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_recenter)">Series GSE272793</a></strong></td>
<td></td>
<td align="right" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_gds)"><a href="/gds/?term=GSE272793[Accession]">Query DataSets for GSE272793</a></td>
</tr></table></td></tr>
<tr valign="top"><td>Status</td>
<td>Public on Sep 06, 2024</td>
</tr>
<tr valign="top"><td nowrap>Title</td>
<td style="text-align: justify">An atlas of small non-coding RNAs in Human Preimplantation Development (Mouse embryo RNA-seq)</td>
</tr>
<tr valign="top"><td nowrap>Organism</td>
<td><a href="/Taxonomy/Browser/wwwtax.cgi?mode=Info&amp;id=10090" onmouseout="onLinkOut('HelpMessage' , geo_empty_help)" onmouseover="onLinkOver('HelpMessage' , geoaxema_organismus)">Mus musculus</a></td>
</tr>
<tr valign="top"><td nowrap>Experiment type</td>
<td>Expression profiling by high throughput sequencing<br></td>
</tr>
<tr valign="top"><td nowrap>Summary</td>
<td style="text-align: justify">Our current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were  found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the inner cell mass (ICM). Additionally, isomiR analyses suggested miRNAs are modified differentially during blastulation. Finally, six novel miRNAs and their embryo gene targets we identified. Our analyses provide the first comprehensive measure of sncRNA biotypes and their corresponding dynamics throughout human preimplantation development, providing an extensive resource. Elucidating the functional roles of sncRNAs during preimplantation development will provide insight into the establishment of the blastocyst and exit from pluripotency.  Further, better understanding the miRNA regulatory programs in human embryogenesis will inform strategies to improve embryo development and outcomes of assisted reproductive technologies. We anticipate broad usage of our data as a resource for studies aimed at understanding embryogenesis, optimising stem cell-based models, assisted reproductive technology, and stem cell biology.<br></td>
</tr>
<tr valign="top"><td nowrap>&nbsp;</td>
<td></td>
</tr>
<tr valign="top"><td nowrap>Overall design</td>
<td style="text-align: justify">Upon reaching the desired developmental stage (8-cell stage at E2.5 or 32-cell stage at E3.5), several control and mimic-treated mouse embryos (3-5 embryos per group per timepoint) were collected and sequenced to investigate the function of miR-381-3p in mouse preimplantation embryosã<br></td>
</tr>
<tr valign="top"><td nowrap>&nbsp;</td>
<td></td>
</tr>
<tr valign="top"><td>Contributor(s)</td>
'''

In [10]:
extract_data(sample_page_html)

'An atlas of small non-coding RNAs in Human Preimplantation Development (Mouse embryo RNA-seq)\nOur current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were  found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the

In [11]:
import time
def data_from_geoid_accession(geo_ids):
    data = []

    joined_ids = ",".join(geo_ids)
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    full_url = f"{base_url}?db=gds&id={joined_ids}&retmode=xml&api_key=f81d6dedf0ae2e62728d7d3290f5359b3f08"

    response = requests.get(full_url)
        

    if response.status_code == 200:
        geo_summaries = (response.text).split('\n\n')
        for geo_summary in geo_summaries:
            accession_code = get_accession(geo_summary)
            page_url = f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={accession_code}'

            page_html = requests.get(page_url)

            if response.status_code == 200:
                to_append = extract_data(page_html.text)
                print(to_append)
                data.append(to_append)
            else:
                print(f"Error accession: {response.status_code}")

    else:
        print(f"Error efetch: {response.status_code}")
    #time.sleep(0.05)

    return data

In [12]:
import xml.etree.ElementTree as ET
def data_from_geoid_without_design(geo_ids):
    joined_ids = ",".join(geo_ids)
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    full_url = f"{base_url}?db=gds&id={joined_ids}&retmode=text&api_key=f81d6dedf0ae2e62728d7d3290f5359b3f08"

    response = requests.get(full_url)
        

    if response.status_code == 200:
        root = ET.fromstring(response.text)
    
        info_list = []

        for doc in root.findall(".//DocSum"):
            entry = ""
            title = doc.find(".//Item[@Name='title']")
            if title is not None:
                entry += (title.text)
                entry += '\n'
        

            summary = doc.find(".//Item[@Name='summary']")
            if summary is not None:
                entry += (summary.text)
                entry += '\n'

            experiment_type = doc.find(".//Item[@Name='gdsType']")
            if experiment_type is not None:
                entry+=(experiment_type.text)
                entry += '\n'
        
            taxon = doc.find(".//Item[@Name='taxon']")
            if taxon is not None:
                entry+=(taxon.text)
                entry += '\n'

            info_list.append(entry)
    else:
        print(f"Error efetch: {response.status_code}")

    return info_list

In [13]:
data_wo_design = data_from_geoid_without_design(['200272793', '200268524', '200263013', '200261729', '200254970'])

In [14]:
data_wo_design

['An atlas of small non-coding RNAs in Human Preimplantation Development (Mouse embryo RNA-seq)\nOur current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the

In [15]:
data = data_from_geoid_accession(geo_ids)

An atlas of small non-coding RNAs in Human Preimplantation Development (Mouse embryo RNA-seq)
Our current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were  found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the i

In [16]:
with open("sample_data.txt", "w") as file:
    for item in data:
        file.write(item + "\n")

In [68]:
data_sample = ['An atlas of small non-coding RNAs in Human Preimplantation Development (Mouse embryo RNA-seq)\nOur current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were  found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the inner cell mass (ICM). Additionally, isomiR analyses suggested miRNAs are modified differentially during blastulation. Finally, six novel miRNAs and their embryo gene targets we identified. Our analyses provide the first comprehensive measure of sncRNA biotypes and their corresponding dynamics throughout human preimplantation development, providing an extensive resource. Elucidating the functional roles of sncRNAs during preimplantation development will provide insight into the establishment of the blastocyst and exit from pluripotency.  Further, better understanding the miRNA regulatory programs in human embryogenesis will inform strategies to improve embryo development and outcomes of assisted reproductive technologies. We anticipate broad usage of our data as a resource for studies aimed at understanding embryogenesis, optimising stem cell-based models, assisted reproductive technology, and stem cell biology.\nMus musculus\nExpression profiling by high throughput sequencing\nUpon reaching the desired developmental stage (8-cell stage at E2.5 or 32-cell stage at E3.5), several control and mimic-treated mouse embryos (3-5 embryos per group per timepoint) were collected and sequenced to investigate the function of miR-381-3p in mouse preimplantation embryosã\x80\x82\n',
 'A robust circulating microRNA signature for diagnosis and early detection in pancreatobiliary cancer\nPurpose: Oncologists for pancreatobiliary cancer (PBca) demand a new circulating biomarker superior to carbohydrate antigen 19-9 (CA19-9). This study aimed to identify the serum microRNA signature composed of reproducible and disease-related microRNAs with clinical bioinformatics.   Methods: This multicenter study enrolled patients with treatment-naÃ¯ve PBca and healthy participants. Optimized serum processing condition was evaluated with t-distributed stochastic neighbor embedding (t-SNE) visualization. The serum microRNA candidates were selected in disease association with Weighted Gene Coexpression Network Analysis (WGCNA). A microRNA signature, combination of multiple serum microRNAs, was examined in exploratory, validation and independent validation set. Biology of the diagnostic miRNAs was evaluated using human pancreatic cancer cells.  Results: The 284 (healthy 150, PBca 134) of 827 serum samples were processed within 2hr of time-to serum collection, distributed at the same area of t-SNE map, assigned to exploratory set. The 193 optimized samples were assigned to validation (healthy 50, PBca 47) or independent validation set (healthy 50, PBca 46). Index-1 was a combination of the 5 serum microRNAs having disease-association on WGCNA, showed a sensitivity/specificity >80% and an AUC outperforming CA19-9 in exploratory, validation and independent validation set. The AUC of Index-1 for detecting T1 tumor was superior to CA19-9 (0.856 vs. 0.649, p = 0.038). Human pancreatic cancer cells had intra- and extracellular expression of miR-665, a component of Index-1. Transfection of miR-665 to a human pancreatic cancer cell inhibited cell growth.   Conclusions: A serum microRNA signature, Index-1, was useful for detecting PBca, which would improve the early diagnosis of PBca.\nHomo sapiens\nNon-coding RNA profiling by array\n284 (healthy 150, PBca 134) serums were analyzed in the training set, and 93 serums were assigned to validation (healthy 50, PBca 47) or independent validation set (healthy 50, PBca 46). Tripricate analyses per a sample.\n',
 'Repetitive element transcript accumulation is related to inflammaging in humans\nInflammation is an established feature of many age-related diseases, and growing evidence implicates transcripts from repetitive elements in inflammatory signaling with aging. We generated transcriptomic (RNA-seq) and whole-genome bisulfite sequencing (WGBS, for methylation) data on frozen/biobanked peripheral blood mononuclear cells from middle-aged and older adults, as well as younger and older adults before and after an exercise intervention, to evaluate correlations among repetitive elements and inflammation, along with methylation of repetitive elements.\nHomo sapiens\nExpression profiling by high throughput sequencingMethylation profiling by high throughput sequencing\nRNA-seq was performed on peripheral blood mononuclear cell (PBMC) samples from 90 healthy, middle age and older adults. RNA-seq and WGBS were also performed on PBMC samples from a separate study of 10 young, healthy sedentary adults, and 13 healthy sedentary older adults before (T1) and after (T2) a 16-week supervised aerobic training intervention.\n',
 'Investigating the transcriptional heterogeneity of the beta cells experiencing Ca2+ excitotoxicity\nBeta cells are remarkably heterogeneous. However, beta cells response to chronic stress at a single cell level remains largely unknown. Here, we used a chemo-genetic model of beta cell excitotoxcity coupled with single cell RNA sequencing to understand beta cells response to chronic stress. Remarkably, we found that a subset of stressed beta cells progressed towards death while the other group evaded the destruction by dedifferentiation. Vulnerable beta cells that progressed towards death showed upregulation of genes implicated in apoptosis, inflammation and immune system activation. The resilient beta cells that evaded the stress showed increased expression aldh1a3, a canonical marker of mammalian beta cell dedifferentiation.\nDanio rerio\nExpression profiling by high throughput sequencing\nWe used three different treatment groups for this study. Tg(ins:Kaede) larvae (n=20) treated with capsaicin (csn) for 48 hours i.e. from 3 to 5 days post fertilization (dpf) were used as control. Tg(ins:TRPV);Tg(ins:Kaede) larvae (n=20) treated with csn for 24 hours i.e. from 4 to 5 dpf were used to identify genes involved in immediate stress response and Tg(ins:TRPV);Tg(ins:Kaede) larvae (n=20) treated with csn for 48 hours i.e. from 3 to 5 dpf were used to identify the genes involved in chronic stress response. Post csn treatment, pancreatic islets from the larvae were manually dissected and then subsequently dissociated via enzymatic digestion. Post dissociation the beta cells were stained with calcein (viability marker dye) and FACS sorted. Individual beta cells were collected in a 384 well plate and their transcriptomes were profiled using smart seq2.RNA_01_H19, RNA_02_G11, RNA_03_P18 do not correspond to the other samples (eg, Ctrl, CNS24, CNS48-2).\n',
 'Age, Sex and Alzheimerâ\x80\x99s disease: a longitudinal study of 3xTg-AD mice reveals sex-specific disease trajectories and inflammatory responses mirrored in postmortem brains from Alzheimerâ\x80\x99s patients\nAging and sex are major risk factors for developing late-onset Alzheimerâ\x80\x99s disease. Compared to men, women are not only nearly twice as likely to develop Alzheimerâ\x80\x99s, but they also experience worse neuropathological burden and cognitive decline despite living longer with the disease. It remains unclear how and when sex differences in biological aging emerge and contribute to Alzheimerâ\x80\x99s disease pathogenesis. We hypothesized that these differences lead to distinct molecular Alzheimerâ\x80\x99s disease signatures in males and females, which could be harnessed for therapeutic and biomarker development. We aged male and female, 3xTg-AD and B6129 control mice across their respective lifespans while longitudinally collecting brain samples. We conducted RNA sequencing analysis on bulk brain tissue and examined differentially expressed genes between 3xTg-AD and B6129 samples and across ages in each sex. 3xTg-AD males experienced an accelerated upregulation of immune-related gene expression in the brain relative to females, especially in genes involved in complement system activation, suggesting distinct inflammatory disease trajectories between the sexes. Our data demonstrate that chronic inflammation and complement activation are associated with increased mortality, revealing that age-related changes in immune response act as a primary driver of sex differences in Alzheimerâ\x80\x99s disease trajectories.\nMus musculus\nExpression profiling by high throughput sequencing\nTo examine molecular changes throughout disease progression, we collected brain tissue from male and female, 3xTg-AD and B6129 control mice, between the ages of 6- and 24-months-old.\n',
 'Epigenetic aging of seminal fluid is associated with inflammation\nMale infertility has been a primary cause of global infertility, affecting 8-12% of couples worldwide. Previous studies have shown that semen quality decreases with advanced aging with an increased presence of inflammatory cells. In this study, we examined the DNA methylation of seminal fluid to understand how the methylome changes with age and infertility. We also compare the age associated changes in semen to those observed in buccal swabs in order to characterize differences in epigenetic aging across diverse tissues. We found that infertility associated changes in the DNA methylation of semen are driven by changes in cell composition, while those associated with aging are linked to inflammatory genes. Many age associated sites are demethylated with advanced aging and are associated with the activation of inflammatory pathways. By contrast, we do not observe age associated changes in inflammatory genes in buccal swab methylomes, which instead are characterized by changes to bivalent promoters. Our study suggests that DNA methylation could serve as a biomarker for male infertility assessment in addition to the standard semen analysis.\nHomo sapiens\nMethylation profiling by high throughput sequencing\nTo examine the changes in DNA methylation of seminal fluids along with age and to compare across tissues, we collected semen and buccal samples from 83 individuals. We then performed targeted bisulfite sequencing on the samples. Finally, we analyzed the DNA methylation data for all individuals.\n',
 'An atlas of small non-coding RNAs in Human Preimplantation Development\nThis SuperSeries is composed of the SubSeries listed below.\nExpression profiling by high throughput sequencingNon-coding RNA profiling by high throughput sequencing\nRefer to individual Series\n',
 'An atlas of small non-coding RNAs in Human Preimplantation Development [Small-Seq]\nOur current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were  found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the inner cell mass (ICM). Additionally, isomiR analyses suggested miRNAs are modified differentially during blastulation. Finally, six novel miRNAs and their embryo gene targets we identified. Our analyses provide the first comprehensive measure of sncRNA biotypes and their corresponding dynamics throughout human preimplantation development, providing an extensive resource. Elucidating the functional roles of sncRNAs during preimplantation development will provide insight into the establishment of the blastocyst and exit from pluripotency.  Further, better understanding the miRNA regulatory programs in human embryogenesis will inform strategies to improve embryo development and outcomes of assisted reproductive technologies. We anticipate broad usage of our data as a resource for studies aimed at understanding embryogenesis, optimising stem cell-based models, assisted reproductive technology, and stem cell biology.\nHomo sapiens\nNon-coding RNA profiling by high throughput sequencing\nWe utilized Small-seq in parallel with Co-seq on single-cells derived from human embryos spanning embryonic day (E) 3 to E7.\n',
 'An atlas of small non-coding RNAs in Human Preimplantation Development [RNA-seq]\nOur current understanding of the molecular circuitries that govern early embryogenesis remains limited, particularly in the human. Small non-coding RNAs (sncRNAs) regulate gene expression transcriptionally and post-transcriptionally, however, the expression of specific biotypes and their dynamics during preimplantation development remains to be determined. Using Small-seq, we identified the abundance of and dynamic expression of piRNA, rRNA, snoRNA, tRNA, and miRNA in human embryos from day 3 to 7. Among the biotypes, both miRNA and snoRNA displayed distinct associations with developmental time and lineage formation, many of which were  found to target key gene expression programs in pluripotency and lineage establishment. We observed an enrichment of the chromosome 19 miRNA cluster (C19MC) in the trophectoderm (TE), and the chromosome 14 miRNA cluster (C14MC) and MEG8-related snoRNAs in the inner cell mass (ICM). Additionally, isomiR analyses suggested miRNAs are modified differentially during blastulation. Finally, six novel miRNAs and their embryo gene targets we identified. Our analyses provide the first comprehensive measure of sncRNA biotypes and their corresponding dynamics throughout human preimplantation development, providing an extensive resource. Elucidating the functional roles of sncRNAs during preimplantation development will provide insight into the establishment of the blastocyst and exit from pluripotency.  Further, better understanding the miRNA regulatory programs in human embryogenesis will inform strategies to improve embryo development and outcomes of assisted reproductive technologies. We anticipate broad usage of our data as a resource for studies aimed at understanding embryogenesis, optimising stem cell-based models, assisted reproductive technology, and stem cell biology.\nHomo sapiens\nExpression profiling by high throughput sequencing\nWe utilized Small-seq in parallel with Co-seq on single-cells derived from human embryos spanning embryonic day (E) 3 to E7.\n',
 'Mapping the chromatin accessibility landscape of zebrafish embryogenesis at single-cell resolution by SPATAC-seq\nZebrafish is a popular model for studies of embryonic development. Recent surveys of zebrafish embryogenesis with single-cell transcriptomics have uncovered hundreds of cell types, but the dynamic accessible regions of the genome that determine regulatory programs responsible for the unique identity and function of each cell type during embryogenesis lack detailed study. Here, we present SPATAC-seq: a split-pool ligation-based high-throughput single-cell assay for transposase-accessible chromatin using sequencing. Using SPATAC-seq, we profiled chromatin accessibility in more than 808,046 individual nuclei across 20 developmental stages spanning the sphere stage to the early larval protruding mouth stages in a single experiment. Using this chromatin accessibility map, we identified 604 cell types and inferred their developmental relationships. We also identified ï»¿959,040 candidate cis-regulatory elements (cCREs) and delineate development-specific cCREs, as well as transcription factors defining diverse cell identities. Importantly, transient reporter assays confirmed that a majority of tested cCREs exhibited robust ï»¿enhanced EGFP expression in restricted cell types or tissues. Finally, we explored gene regulatory programs that drive pigment and notochord cell differentiation. Our work provides a comprehensive and valuable open resource for exploring transcriptional regulators that determine cell fate specification in zebrafish embryogenesis.\nDanio rerio\nGenome binding/occupancy profiling by high throughput sequencing\nWild-type zebrafish strain TÃ¼bingen (TU) line were maintained in standard zebrafish units at Core Facilities, school of life sciences, Tsinghua University. Naturally spawned fish embryos were collected at the 1-cell stage and the time of fertilization was used to stage each clutch. Embryos were maintained in Holtfreterâ\x80\x99s solution at 28.5 Â°C for time course experiments and processed for bulk ATAC-seq and SPATAC-seq at the indicated times. Embryo nuclei of 20 developmental stages were extracted via previously described methods (Corces MR, et al. Nat Methods. 2017; Cao J, et al. Nature. 2019). The isolated nuclei were used for bulk ATAC-seq with TruePrep DNA Library Prep Kit V2 for Illumina (TD501, Vazyme), SPATAC-seq and 10x Genomics scATAC-seq (See more detailed method procedures from our paper). SPATAC-seq and bulk ATAC-seq libraries were sequenced with paired-end 150-bp reads on Illumina NovaSeq 6000 platform or DNBSEQ-T7. 10x Genomics scATAC-seq were sequenced with paired-end 100-bp reads on MGISEQ-2000RS, and i5 16 bp were linked at the end of Read 2.Please note that the Series supplementary *bw files were genearated from all 535 samples together.\n',
 'Age-related changes in human skeletal muscle transcriptome and proteome are more affected by chronic inflammation and physical inactivity than primary aging\nEvaluation of the influence of primary and secondary aging on the manifestation of molecular and cellular hallmarks of aging is a challenging and currently unresolved issue. Our study represents the first demonstration of the distinct role of primary aging and both chronic inflammation and physical inactivity â\x80\x93 the most important drivers of secondary aging, in the regulation of transcriptomic and proteomic profiles in human skeletal muscle. To achieve this purpose, young healthy people (n=15), young (n=8) and older (n=37) patients with knee/hip osteoarthritis, a model to study the effect of long-term inactivity and chronic inflammation on the vastus lateralis muscle, were included in the study. It was revealed that widespread and substantial age-related changes in gene expression (~4,000 genes regulating mitochondrial function, proteostasis, cell membrane, secretory and immune response) were related to the long-term physical inactivity and chronic inflammation rather than primary aging. Primary aging contributed mainly to the regulation of genes (~200) encoding nuclear proteins (regulators of DNA repair, RNA processing, and transcription), mitochondrial proteins (genes encoding respiratory enzymes, mitochondrial complex assembly factors, regulators of cristae formation and mitochondrial reactive oxygen species production), as well as regulators of proteostasis. It was found that proteins associated with aging were regulated mainly at the post-transcriptional level. The set of putative primary aging genes and their potential transcriptional regulators can be used as a resource for further targeted studies investigating the role of individual genes and related transcription factors in the emergence of a senescent cell phenotype.\nHomo sapiens\nExpression profiling by high throughput sequencing\nThe study involved 37 older patients with advancedâ\x80\x90stage, symptomatic knee/hip osteoarthritis (OP; median age 72 years and interquartile range [69â\x80\x9377] years, M:F = 5:32), 8 young patients with the same diagnosis (YP; 39 [37â\x80\x9342] years, M:F = 7:1), and 15 young healthy volunteers (YH; 35 [28â\x80\x9338] years, M:F = 13:2). Biopsies of muscle tissue from m. vastus lateralis were taken after an overnight fast, placed in the Custodiol buffer (Dr. Franz KÃ¶hler Chemie GmbH, Germany) and processed within 10â\x80\x9315 minutes after the biopsy. Thus, visible fragments of connective and adipose tissue were removed; a part of the tissue was fixed for histological examination, and another part was frozen in liquid nitrogen and stored at -80 Â°C.\n',
 'BL6Mousei: MSL2 is an allelic dosage sensor in mammals\nAllele-specific gene expression have been observed in genomic imprinting, X chromosome inactivation, allelic exclusion of immunoglobulin loci and non-canonical random monoallelic expression. Here we show that MSL2 can regulate allele specific gene expression and we elucidate its mechanism of action. MSL2 is known to regulate gene dosage up to two fold in drosophila and mammals. In addition to its canonical role as a milder modulator, we found that the loss of MSL2 switches off gene expression completely in an allele specific way and results in biallelic to monoallelic gene expression, which will be ignored as classical down regulated genes without applying the mouse hybrid F1 system. It indicates that MSL2 can activate the expression of the allele which is otherwise inactive intrinsically. We further found this is achieved by the interplay between MSL and KANSL complex. MSL2 acts as a methylation-insensitive factor, which can prevent DNA methylation of the inactive allele, thus facilitating occupancy of methylation-sensitive factor KANSL3 and activating the gene expression. The novel mechanism of MSL2 provides a safeguarding system to warrant biallelic gene expression of intrinsically monoallelic expressed genes including X-inactivated and imprinting genes. These genes are further discovered as sex bias genes such as X-inactivation escaped genes (escapees) are expressed higher in females. Furthermore, the biallelic expression of MSL2 regulated escapees may protect females from single loss of function mutation and contribute to male severer or predominance neurodevelopmental diseases.\nMus musculus\nExpression profiling by high throughput sequencing\nAllele specific differential analysis of various sequencing data from ESCs and NPCs of female and male hybrid clones.\n',
 'Transcriptome analysis of human iPSC-neurons treated with cardiac glycosides\nTo investigate the molecular mechanisms of cardiac glycosidesâ\x80\x99 neuroprotection, we profiled transcriptomes of human iPSC-neurons after 72h of treatment with increasing doses of digoxin, oleandrin, proscillaridin A or vehicle alone (0.1% DMSO) using RNA sequencing.\nHomo sapiens\nExpression profiling by high throughput sequencing\nThe NPC line MGH-2046-RC1 (P301L) was derived from a female individual in her 50s with FTD carrying the autosomal dominant mutation P301L (c.C1907T NCBI NM_001123066, rs63751273), and differentiated into neuronal cells over 6-8 weeks by growth factor withdrawal. The iPSC-neurons were treated with increasing doses of digoxin, oleandrin, proscillaridin A or vehicle alone (0.1% DMSO) for 72h, followed by RNA-seq.\n',
 'CaBlCaXi: MSL2 is an allelic dosage sensor in mammals\nAllele-specific gene expression have been observed in genomic imprinting, X chromosome inactivation, allelic exclusion of immunoglobulin loci and non-canonical random monoallelic expression. Here we show that MSL2 can regulate allele specific gene expression and we elucidate its mechanism of action. MSL2 is known to regulate gene dosage up to two fold in drosophila and mammals. In addition to its canonical role as a milder modulator, we found that the loss of MSL2 switches off gene expression completely in an allele specific way and results in biallelic to monoallelic gene expression, which will be ignored as classical down regulated genes without applying the mouse hybrid F1 system. It indicates that MSL2 can activate the expression of the allele which is otherwise inactive intrinsically. We further found this is achieved by the interplay between MSL and KANSL complex. MSL2 acts as a methylation-insensitive factor, which can prevent DNA methylation of the inactive allele, thus facilitating occupancy of methylation-sensitive factor KANSL3 and activating the gene expression. The novel mechanism of MSL2 provides a safeguarding system to warrant biallelic gene expression of intrinsically monoallelic expressed genes including X-inactivated and imprinting genes. These genes are further discovered as sex bias genes such as X-inactivation escaped genes (escapees) are expressed higher in females. Furthermore, the biallelic expression of MSL2 regulated escapees may protect females from single loss of function mutation and contribute to male severer or predominance neurodevelopmental diseases.\nMus musculus\nExpression profiling by high throughput sequencing\nAllele specific differential analysis of various sequencing data from ESCs and NPCs of female and male hybrid clones.\n',
 'Xenografted human microglia display diverse transcriptomic states in response to Alzheimerâ\x80\x99s disease-related AÎ² pathology\nMicroglia are central players in Alzheimerâ\x80\x99s Disease (AD) pathology, but analyzing microglia states in human brain samples is challenging due to genetic diversity, postmortem delay and admixture of pathologies. To circumvent these issues, here we generated 138,577 single cell expression profiles of human stem cell-derived microglia xenotransplanted in the brain of the AppNL-G-F model of amyloid pathology and wild type controls. Xenografted human microglia adopt a disease-associated (DAM) profile similar to that seen in mouse microglia, but display a more pronounced HLA state, likely related to antigen presentation in response to amyloid plaques. The human microglial response also involves a pro-inflammatory cytokine/chemokine CRM response to oligomeric AÎ²o. Genetic deletion of TREM2 or APOE, as well as APOE polymorphisms and TREM2R47H expression in the transplanted microglia modulate these responses differentially. The expression of other AD risk genes is differentially regulated across the distinct cell states elicited in response to amyloid pathology. Thus, we have identified multiple transcriptomic cell states adopted by human microglia in response to AD-related pathology, which should be taken into account in translational studies.\nHomo sapiens\nExpression profiling by high throughput sequencing\nThe main goal of this project is to decipher the complex human microglial response towards amyloid aggregates. Here, we transplanted human microglia progenitors from several stem cell lines (UKBIO11-A, BIONi010-C and H9) into the mouse brain of AppNL-G-F, Apphu/hu or AppNL-G-F ApoE-/-  mice to mimic several amyloid pathology conditions, including injection of oligomeric AÎ²o. After exposure to diseased conditions, we isolated human microglia and sequenced more than 138,000 microglia across 106 mice.\n',
 'Small Molecule Inducers of Neuroprotective miR-132 Identified by HTS-HTS in Human iPSC-derived Neurons\nMicroRNAs (miRNAs) are short RNAs that regulate fundamental biological processes. miR-132, a key miRNA with established functions in Tau homeostasis and neuroprotection, is consistently downregulated in Alzheimerâ\x80\x99s disease (AD) and other tauopathies. miR-132 overexpression rescues neurodegenerative phenotypes in several AD models. To complement research on miRNA-mimicking oligonucleotides targeting the central nervous system, we developed a high-throughput-screen coupled high-throughput-sequencing (HTS-HTS) in human induced pluripotent stem cell (iPSC)-derived neurons to identify small molecule inducers of miR-132. We discovered that cardiac glycosides, which are canonical sodium-potassium ATPase inhibitors, selectively upregulated miR-132 in the sub-Î¼M range. Coordinately, cardiac glycoside treatment downregulated total and phosphorylated Tau in rodent and human neurons and protected against toxicity by glutamate, N-methyl-D-aspartate, rotenone, and AÎ² oligomers. In conclusion, we identified small-molecule drugs that upregulated the neuroprotective miR-132 and ameliorated neurodegenerative phenotypes. Our dataset also represents a comprehensive resource for discovering small molecules that modulate specific miRNAs for therapeutic purposes.\nHomo sapiens\nNon-coding RNA profiling by high throughput sequencing\nHuman NGN2-driven iPSC-derived neurons (NGN2-iNs) were cultured in 96-well plates. On day 19, the Selleckchem library (N=1,902 compounds), a diverse library of bioactive molecules, was pin-transferred into plates to achieve 10 Î¼M final concentration. DMSO (0.1% final concentration) and forskolin (10 Î¼M) were used as the negative and positive controls, respectively. NGN2-iNs were imaged to monitor neuronal health 24h later, followed by direct lysis to release RNA and RealSeq small RNA library preparation. Illumina sequencing was performed in six batches, each indexed with 384 multiplex barcodes. Exclusion criteria included cell death, neurite degeneration, loss of cells during washes, enrichment of astrocytes, or less than 1,000 total annotated read counts.\n',
 'The essential roles of FXR in diet and aging affected metabolic changes and liver disease development: a multi-omics study\nAging and unhealthy diets are risks for metabolic diseases including liver cancer. Bile acid receptor farnesoid X receptor (FXR) knockout (KO) mice develop metabolic liver diseases and progress into liver cancer as they age, and Western diet (WD) intake facilitates liver carcinogenesis in those mice. This study aimed to uncover molecular signatures within the gut-liver axis for diet and age-linked liver diseases in FXR-dependent or independent manners.Many more transcripts were changed due to WD intake and aging in WT mice than those in FXR KO mice. In other words, WD intake and aging impact the hepatic transcriptomes and metabolomes in an FXR-dependent manner. In WT mice, WD/aging upregulated inflammation-related genes and downregulated genes involved in oxidative phosphorylation (OXPHOS). Urine metabolomes provided clear distinguishing for differential dietary intake. By contrast, the metabolomes of the liver, serum, or urine could reflect age differences. Further, irrespective of differential diets intake or ages, transcriptomes, metabolomes, and cecal microbiota distinguished WT and FXR KO. Western dietary patterns and aging share molecular commonality with FXR deactivation. Notably, WD, aging, and FXR deactivation commonly altered hepatic cell division-related transcripts (Cenpe, Ect2, Top2a, Kif20a, Tpx2, Nuf2, Kif18b, Aspm, E2f8, and Hmmr), which are associated with overall survival rate in HCC patients. In conclusion, FXR is essential for maintaining metabolic homeostasis in response to WD intake and aging. FXR activation helps to alleviate diet and/or aging-induced metabolic health issues.\nMus musculus\nExpression profiling by high throughput sequencing\nWild-type and Farnesoid X receptor (FXR) KO mice were fed a control diet (CD) or Western diet (WD) since weaning. Transcriptome profile of livers from these mice at the age of 5, 10, and 15 months (4 mice per group).\n',
 'Precision pharmacological reversal of strain-specific diet-induced metabolic syndrome in mice informed by epigenetic and transcriptional regulation\nWe exposed three mouse strains to a high-fat high-carbohydrate diet, leading to varying degrees of metabolic syndrome. Comprehensive methylation and transcriptomic analysis found overlapping but also highly divergent changes in DNA methylation and gene expression, with pathway analysis suggesting a strategy for strain-specific targeted pharmacologic intervention of upstream regulators. Administeration of the drug GW4064 to target one of these genotype-dependent networks, the farnesoid X receptor pathway resulted in genotype-specific protection against dietary effects in BL6 as predicted by our epigenetic analysis.\nMus musculus\nMethylation profiling by high throughput sequencingExpression profiling by high throughput sequencing\nWGBS and RNA-Seq was performed on BL6, A/J, and NOD mice fed Standard or American diets. Some mice were also administered DMSO vehicle or the FXR agonist GW4064.\n',
 "SPATAC-seq: split-pool ligation-based single cell ATAC-seq method\nTo establish an ultra-high-throughput single cell chromatin accessibility profiling method that is cost-effective and widely accessible, we built on sci-ATAC-seq (Cusanovich, D. A. et al. Science. 2015; Amini, S. et al. Nature Genetics. 2014) and SPLIT-seq (Rosenberg, A. B. et al. Science. 2018) to design SPATAC-seq, which in situ label chromatin fragment in the same single cell through combinatorial barcoding. Briefly, in SPATAC-seq, (1) fixed nuclei are transposed in 48 different wells by 48 unique Tn5 transposase, which containing barcoded adaptors and 5'-phosphorylation; (2) the nuclei from all wells are collected and redistributed into second and third 48-well plate in turn, where the next two rounds of indexing are achieved through into either end of the custom transposome, which result in the generation of more than 110,000 (48^3) unique barcode combinations. (3) the nuclei are pooled, split into sublibraries and lysed, and the DNA was amplified by polymerase chain reaction (PCR), which introduce illumina sequencing barcodes and complete libraries construction. (4) After sequencing, fastq files were demultiplexed according to the same four-barcode combinations. For profiling more cells in one sublibrary, we can increase the number of barcode combinations by increase the number of indexing of each round to 96, which can produce about 1 million combinations. To assess the fidelity of SPATAC-seq, we performed a species-mixing experiment with cultured human (K562) and mouse (Hepa) cells.  Here, we tagged mixed permeabilized nuclei with only 8 barcoded transposome. In round 4, we generated eight sublibraries with different cell-recovery targets, which can be used to evaluating the stability of this method and the correlation between real doublet rates and theoretical value.\nGenome binding/occupancy profiling by high throughput sequencing\nWe firstly assembled 48 indexed Tn5 transposome complexes, and then performed a species-mixing experiment with cultured human (K562) and mouse (Hepa) cells to prove the feasibility of SPATAC-seq.  Here, we tagged mixed permeabilized nuclei with only 8 barcoded transposome, and generated eight sublibraries with different cell-recovery targets.\n",
 'DNA methylation restricts default deterministic fates towards neural and germline lineages\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nExpression profiling by high throughput sequencingMethylation profiling by high throughput sequencingGenome binding/occupancy profiling by high throughput sequencing\nRefer to individual Series\n',
 'DNA methylation restricts default deterministic fates towards neural and germline lineages during early development\nWe investigate the impact of DNA methylation on chromatin accessibility during epiblast differentiation. To this end, we used in vitro Epiblast-like cell (EpiLC) differentiation on WT and DNA-methylation free TKO murine ESCs. We next performed ATAC seq using TDE1 Tn5 transposase (Nextera).\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencing\nATAC-seq on WT and TKO ESCs (D0), and D2, D4 EpiLCs\n',
 'Methylation data from stored peripheral blood leukocytes from African American participants in the GENOA study\nThis SuperSeries is composed of the SubSeries listed below.\nHomo sapiens\nMethylation profiling by genome tiling array\nRefer to individual Series\n',
 'Methylation data from stored peripheral blood leukocytes from African American participants in the GENOA study with Infinium HumanMethylationEPIC BeadChip\nAfrican-American individuals of the GENOA study\nHomo sapiens\nMethylation profiling by genome tiling array\nThe Genetic Epidemiology Network of Arteriopathy (GENOA) study is a community-based study of hypertensive sibships that was designed to investigate the genetics of hypertension and target organ damage. The study includes African Americans (AA) from Jackson. In the initial phase of GENOA (Phase I: 1996-2001), all members of sibships containing at least 2 individuals with essential hypertension clinically diagnosed before age 60 were invited to participate, including both hypertensive and normotensive siblings. The exclusion criteria for GENOA included secondary hypertension, alcoholism or drug abuse, pregnancy, insulin-dependent diabetes mellitus, or active malignancy. Eighty percent of AA (N=1,482) from the initial study population returned for the second examination (Phase II: 2001-2005). Demographic information, medical history, clinical characteristics, lifestyle factors, and blood samples were collected in each phase. Written informed consent was obtained from all subjects and approval was granted by participating institutional review boards (University of Michigan, University of Mississippi Medical Center, and Mayo Clinic). Genomic DNA was extracted from stored peripheral blood leukocytes that was collected at Phase 1 (N=1106) or Phase 2 (N=304) using AutoGen FlexStar (AutoGen, Holliston, MA). Bisulfite conversion was performed with the EZ DNA Methylation Kit (Zymo Research, Irvine, CA), and methylation was assessed using the Illumina Infinium HumanMethylationEPIC BeadChip. In total 1,394 samples remained after quality control and filtering.\n',
 'Methylation data from stored peripheral blood leukocytes from African American participants in the GENOA study with Infinium HumanMethylation450 BeadChip\nAfrican-American individuals of the GENOA study\nHomo sapiens\nMethylation profiling by genome tiling array\nThe Genetic Epidemiology Network of Arteriopathy (GENOA) study is a community-based study of hypertensive sibships that was designed to investigate the genetics of hypertension and target organ damage. The study includes African Americans (AA) from Jackson. In the initial phase of GENOA (Phase I: 1996-2001), all members of sibships containing at least 2 individuals with essential hypertension clinically diagnosed before age 60 were invited to participate, including both hypertensive and normotensive siblings. The exclusion criteria for GENOA included secondary hypertension, alcoholism or drug abuse, pregnancy, insulin-dependent diabetes mellitus, or active malignancy. Eighty percent of AA (N=1,482) from the initial study population returned for the second examination (Phase II: 2001-2005). Demographic information, medical history, clinical characteristics, lifestyle factors, and blood samples were collected in each phase. Written informed consent was obtained from all subjects and approval was granted by participating institutional review boards (University of Michigan, University of Mississippi Medical Center, and Mayo Clinic). Genomic DNA was extracted from stored peripheral blood leukocytes that was collected at Phase-1 (N=422) using AutoGen FlexStar (AutoGen, Holliston, MA). Bisulfite conversion was performed with the EZ DNA Methylation Kit (Zymo Research, Irvine, CA), and methylation was assessed using the Illumina Infinium HumanMethylation450 BeadChip. In total 418 samples remained after quality control and filtering.\n',
 'Characterization and analysis of the promoter region of Monodehydroascorbate reductase 4 (CpMDAR4) in papaya\nThe aim of this research was to study the effects of cis-element differences between the X, Y and Yh alleles on the expression of CpMDAR4, a potential candidate gene for sex differentiation in papaya, using a transcriptional reporter system in a model species Arabidopsis thaliana. Possible effects of a retrotransposon insertion in the Y and Yh alleles on the transcription and expression of CpMDAR4 alleles in papaya flowers were also examined. When comparing promoters and cis-regulatory elements among genes in the non-recombining region of the sex chromosomes, paired genes exhibited differences. Our results showed that differences in the promoter sequences of the CpMDAR4 alleles drove the expression of a reporter gene to different flower tissues in Arabidopsis. Î²-glucuronidase staining analysis of T2 and T3 lines for constructs containing 5â\x80\x99 deletions of native Y and Yh allele promoters showed the loss of specific expression of the reporter gene in the anthers, confirming the existence and location of cis-regulatory element POLLEN1LELAT52. The expression analysis of CpMDAR4 alleles in papaya flowers also showed that all alleles are actively expressed in different flower tissues, with the existence of a shorter truncated isoform, with unknown function, for the Y and Yh alleles due to an LTR-RT insertion in the Y and Yh chromosomes. The observed expression patterns in Arabidopsis thaliana flowers and the expression patterns of CpMDAR4 alleles in papaya flowers, suggest that MDAR4 might have a role on development of reproductive organs in papaya, and that it constitutes an important candidate for sex differentiation.\nCarica papaya\nExpression profiling by high throughput sequencing\nTo evaluate tissue-specific expression of CpMDAR4 alleles in papaya flowers, total RNA was extracted from small flower buds (< 1 mm), reproductive organs of pre-meiotic (1-6 mm) and post-meiotic (7-12 mm) flower buds (including aborted pistils or â\x80\x9cpistillodeâ\x80\x9d), mature flowers (15-20 mm) and gametophytes of mature â\x80\x98AU9â\x80\x99 female flowers and â\x80\x98AU9â\x80\x99 male flowers.\n',
 'DNA methyltransferases 3A and 3B target specific sequences during mouse gastrulation\nWe performed single-cell transcriptome analysis (using MARS-seq) of mouse embryonic stem cells, embryiod bodies and mouse embryos. Additionally, we performed methylation analysis of bulk and single cells of this source (using whole-genome PBAT and PBAT capture).\nMus musculus\nExpression profiling by high throughput sequencingMethylation profiling by high throughput sequencing\nThe ESCs lines that were used are J1 (SCRC1010, ATCC), Dnmt3a-/- clone 6aa, Dnmt3b-/- clone 8bb, Dnmt1-/- Dnmt3a -/- Dnmt3b-/- clone 19 (TKO) (Cell Bank Riken BioResource Research Center) , Dnmt3a -/- Dnmt3b-/- clones 7aabb and 16aabb (DKO) (kindly provided by Dr. Masaki Okano), Wild type,  Dnmt3a and Dnmt3b knockout ES line, passages 3 or 5 in 2i/L, respectively (N15, kindly provided by Dr. Masaki Yagi). Embryiod bodies were generated by hanging drop protocol . Cells  and embryos were collected dissociated  and FACS-sorted into single-cell methylation or scRNA profiling or were collected as a bulk a long time points.\n',
 'Metabolic competition between lipid metabolism and histone methylation regulates sexual differentiation in human malaria parasites\nLike most pathogens, malaria parasites balance persistence in the current host with transmission to the next. For Plasmodium falciparum, the most widespread and virulent human malaria parasite, persistence depends on continuous asexual replication in red blood cells while transmission requires differentiation into non-replicating gametocytes, the male and female cells able to infect the mosquito vector. This developmental decision is controlled via stochastic derepression of a heterochromatin-silenced locus encoding the transcription factor PfAP2-G, which acts as the master switch for sexual differentiation. Recent work showed that the frequency of pfap2-g derepression is responsive to extracellular levels of phospholipid precursors. However, the regulatory mechanisms linking these metabolites to epigenetic regulation of pfap2-g was hitherto unknown. Using chemical, metabolomic, and genetic approaches, we show that this response is mediated by metabolic competition for the methyl donor S-adenosylmethionine between histone methyltransferases and phospho-ethanolamine methyltransferase, a critical enzyme in the parasiteâ\x80\x99s pathway for de novo phosphatidyl-choline synthesis. When phosphatidyl-choline precursors are scarce, increased consumption of S-adenosylmethionine for de novo synthesis of phosphatidyl-choline results in a decrease in histone methylation marks that mediate silencing of pfap2-g, thereby up-regulating its transcription and increasing the frequency of sexual differentiation. Our findings reveal a key connection between metabolite utilization and gene expression in malaria parasites that forms the mechanistic link between phosphatidyl-choline metabolism and the frequency of sexual commitment.\nPlasmodium falciparum\nGenome binding/occupancy profiling by high throughput sequencing\nWe performed Cut&RUN for H3K9me3 and H3K4me3 on P. falciparum nuclei under different growth media conditions\n',
 'A gene silencing screen uncovers diverse tools for targeted gene repression in Arabidopsis\nDNA methylation has been utilized for target gene silencing in plants, however itâ\x80\x99s not well-understood whether other silencing pathways can be also used to manipulate gene expression. Here we performed a gain-of-function screen for proteins that could silence a target gene when fused to an artificial zinc finger. We uncovered many proteins that suppressed gene expression through DNA methylation, histone H3K27me3 deposition, H3K4me3 demethylation, histone deacetylation, inhibition of RNA Polymerase II transcription elongation or Ser-5 dephosphorylation. These proteins also silenced many other genes with different efficacy, and a machine learning model could accurately predict the efficacy of each silencer based on various chromatin features of the target loci. Furthermore, some proteins were also able to target gene silencing when used in a dCas9-SunTag system. These results provide a more comprehensive understanding of epigenetic regulatory pathways in plants and provide an armament of tools for targeted gene manipulation.\nArabidopsis thaliana\nGenome binding/occupancy profiling by high throughput sequencingExpression profiling by high throughput sequencingMethylation profiling by high throughput sequencing\nThis dataset includes ChIP-seq, RNA-seq and BS-PCR to understand the molecular mechanism of Arabidopsis ZF targeting proteins.\n',
 "A skeletal-muscle senescence blueprint defines an aged-like inflamed niche that inhibits regeneration over lifetime [scRNA-seq]\nA new sorting protocol was used to identify and isolate different senescent cell types from damaged muscles of young and geriatric mice. Analysis revealed conservation of two universal senescence hallmarks (inflammation and fibrosis) across cell type, regeneration time, and aging, while cell identity traits were preserved. Senescent cells create an â\x80\x9caged-likeâ\x80\x9d inflamed niche, mirroring inflammation-associated with aging (inflammaging), that arrests stem cell proliferation and regeneration.\nMus musculus\nExpression profiling by high throughput sequencing\nMuscles were mechanically disaggregated and incubated in Dulbecco's Modified Eagle's medium (DMEM) containing liberase (Roche) and dispase at 37Â°C with agitation for 1-2 hours. When required, SPiDER-b-gal reagent (Dojindo) was added during the second hour. The supernatant was then filtered and cells were incubated in lysis buffer (BD Pharm Lyse) for 10 min on ice, resuspended in PBS with 2.5% fetal bovine serum (FBS), and counted. BV711-conjugated anti- CD45 (BD), APC-Cy7-conjugated anti F4/80 (Biolegend), PE- conjugated anti-a7-integrin (Ablab), APC-conjugated anti-CD31 (eBioscience) and PE-Cy7-conjugated anti-Sca-1 (Biolegend) antibodies were used to isolate MCs (CD45+ and F4/80+), SCs (a7-integrin+, CD45+, F4/80+ and CD31+) and FAPs (Sca-1+, MCs CD45+, F4/80+, a7-integrin+ and CD31+). PE-Cy7-conjugated anti-CD45 antibody (Biolegend) was used to isolate CD45 positive and negative populations. SPiDER-b-gal (SPiDER) was employed to isolate senescent cells (SPiDER+) from non-senescent cells (SPiDERâ\x80\x93) of each cell type. Isolated cells were subjected to single-cell RNA sequencing. 1 and 2 are biological replicates.\n",
 'A skeletal-muscle senescence blueprint defines an aged-like inflamed niche that inhibits regeneration over lifetime\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nExpression profiling by high throughput sequencingGenome binding/occupancy profiling by high throughput sequencing\nRefer to individual Series\n',
 "A skeletal-muscle senescence blueprint defines an aged-like inflamed niche that inhibits regeneration over lifetime [ATAC-seq]\nA new sorting protocol was used to identify and isolate different senescent cell types from damaged muscles of young and geriatric mice. Analysis revealed conservation of two universal senescence hallmarks (inflammation and fibrosis) across cell type, regeneration time, and aging, while cell identity traits were preserved. Senescent cells create an â\x80\x9caged-likeâ\x80\x9d inflamed niche, mirroring inflammation-associated with aging (inflammaging), that arrests stem cell proliferation and regeneration.\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencing\nMuscles were mechanically disaggregated and incubated in Dulbecco's Modified Eagle's medium (DMEM) containing liberase (Roche) and dispase at 37Â°C with agitation for 1-2 hours. When required, SPiDER-b-gal reagent (Dojindo) was added during the second hour. The supernatant was then filtered and cells were incubated in lysis buffer (BD Pharm Lyse) for 10 min on ice, resuspended in PBS with 2.5% fetal bovine serum (FBS), and counted. BV711-conjugated anti- CD45 (BD), APC-Cy7-conjugated anti F4/80 (Biolegend), PE- conjugated anti-a7-integrin (Ablab), APC-conjugated anti-CD31 (eBioscience) and PE-Cy7-conjugated anti-Sca-1 (Biolegend) antibodies were used to isolate MCs (CD45+ and F4/80+), SCs (a7-integrin+, CD45+, F4/80+ and CD31+) and FAPs (Sca-1+, MCs CD45+, F4/80+, a7-integrin+ and CD31+). PE-Cy7-conjugated anti-CD45 antibody (Biolegend) was used to isolate CD45 positive and negative populations. SPiDER-b-gal (SPiDER) was employed to isolate senescent cells (SPiDER+) from non-senescent cells (SPiDERâ\x80\x93) of each cell type. Isolated cells were subjected to ATAC-seq. Pools of 3 animals were used for each sample.\n",
 "A skeletal-muscle senescence blueprint defines an aged-like inflamed niche that inhibits regeneration over lifetime [RNA-seq]\nA new sorting protocol was used to identify and isolate different senescent cell types from damaged muscles of young and geriatric mice. Analysis revealed conservation of two universal senescence hallmarks (inflammation and fibrosis) across cell type, regeneration time, and aging, while cell identity traits were preserved. Senescent cells create an â\x80\x9caged-likeâ\x80\x9d inflamed niche, mirroring inflammation-associated with aging (inflammaging), that arrests stem cell proliferation and regeneration.\nMus musculus\nExpression profiling by high throughput sequencing\nMuscles were mechanically disaggregated and incubated in Dulbecco's Modified Eagle's medium (DMEM) containing liberase (Roche) and dispase at 37Â°C with agitation for 1-2 hours. When required, SPiDER-b-gal reagent (Dojindo) was added during the second hour. The supernatant was then filtered and cells were incubated in lysis buffer (BD Pharm Lyse) for 10 min on ice, resuspended in PBS with 2.5% fetal bovine serum (FBS), and counted. BV711-conjugated anti- CD45 (BD), APC-Cy7-conjugated anti F4/80 (Biolegend), PE- conjugated anti-a7-integrin (Ablab), APC-conjugated anti-CD31 (eBioscience) and PE-Cy7-conjugated anti-Sca-1 (Biolegend) antibodies were used to isolate MCs (CD45+ and F4/80+), SCs (a7-integrin+, CD45+, F4/80+ and CD31+) and FAPs (Sca-1+, MCs CD45+, F4/80+, a7-integrin+ and CD31+). PE-Cy7-conjugated anti-CD45 antibody (Biolegend) was used to isolate CD45 positive and negative populations. SPiDER-b-gal (SPiDER) was employed to isolate senescent cells (SPiDER+) from non-senescent cells (SPiDERâ\x80\x93) of each cell type. Isolated cells were subjected to low-input RNA sequencing. Pools of 3 animals were used for each sample.File name legend: We performed bulk RNA-seq analysis from young (labeled as Y) or old mice (labeled as G), senescent cells (labeled as Sen), and non-senescent cells (labeled as NSen) and NSen cells from basal (contralateral) muscles (with no further label) from three lineages: Satellite cell lineage (labeled as S), Fibroadipogenic progenitors  (labeled as F), and myeloid cells (labeled as M) at early (3 d.p.i. - 3) and late (7 d.p.i.- 7) phases of regeneration. Biological replicates are indicated with a number.Example:Y3SNSen_1 Young 3 d.p.i Non-senescent cells, biological replicate 1Y3SSen_1 Young 3 d.p.i Senescent cells, biological replicate 1Y3S_1 Young Non-senescent cells from basal (contralateral) muscles of the 3 days time point, biological replicate 1\n",
 'Circadian alignment of early onset caloric restriction promotes longevity in male C57BL/6J mice\nCaloric restriction (CR) prolongs lifespan, yet the mechanisms by which it does so remain poorly understood. Under CR, mice self-impose chronic cycles of 2-hour-feeding and 22-hour-fasting, raising the question whether calories, fasting, or time of day are causal. We show that 30%-CR is sufficient to extend lifespan 10%; however, a daily fasting interval and circadian-alignment of feeding act together to extend lifespan 35% in male C57BL/6J mice. These circadian effects of CR are independent of fasting duration and body weight. Aging induces widespread increases in gene expression associated with inflammation and decreases in expression of genes encoding components of metabolic pathways in liver from ad lib fed mice. CR at night ameliorates these aging-related changes. Thus, circadian interventions promote longevity and provide a perspective to further explore mechanisms of aging.\nMus musculus\nExpression profiling by high throughput sequencing\nmRNA-seq circadian profiles generated for C57BL/6J male mice fed under 6 different feeding conditions and collected at 6 and 19 months of age. Liver samples were collected every 4h during 2 days (12 timepoints x 2 biological replicates x 6 feeding conditions x 2 ages, total 288 samples).\n',
 'Stromal inflammation is a targetable driver of blood aging [droplet-based scRNAseq - Batch 2]\nTo investigate the role of IL1 signaling in hematopoietic aging, we performed droplet-based scRNAseq (10X Genomics) of hematopoietic stem and progenitor cells (LK/LSK) and unfractionated endosteal and central marrow stromal cells (Ter119-/CD45-) from young wild type and young and old wild type and IL1R1-/- C57BL/6 mice.\nMus musculus\nExpression profiling by high throughput sequencing\nHematopoietic stem and progenitor cells (lineage negative, cKit positive (LK) and lineage negative, cKit positive, Sca-1 positive (LSK)) and endosteal and central marrow stromal cells from the bone marrow of young (10 weeks) and old (24 months) wild-type C57BL/6 mice were isolated by flow cytometry and subjected to droplet-based scRNAseq (10X Genomics).\n',
 'Dynamics of DNA hydroxymethylation and methylation during mouse embryonic and germline development\nIn mammals, DNA 5-hydroxymethylcytosine (5hmC) is involved in methylation reprogramming during early embryonic development. Yet, to what extent 5hmC participates in genome-wide methylation reprogramming remains largely unknown. Here, we characterize the 5hmC landscapes in mouse early embryos and germ cells with parental allele specificity. DNA hydroxymethylation was most strongly correlated with DNA demethylation as compared with de novo or maintenance methylation in zygotes, while 5hmC was targeted to particular de novo methylated sites in postimplantation epiblasts. Surprisingly, DNA replication was also required for 5hmC generation, especially in the female pronucleus. More strikingly, aberrant nuclear localization of Dnmt1/Uhrf1 in mouse zygotes due to maternal deficiency of Nlrp14 led to defects in DNA-replication-coupled passive demethylation and impaired 5hmC deposition, revealing the divergency between genome-wide 5-methylcytosine (5mC) maintenance and Tet-mediated oxidation. In summary, our work provides insights and a valuable resource for the study of epigenetic regulation in early embryo development.\nMus musculus\nMethylation profiling by high throughput sequencingExpression profiling by high throughput sequencing\nACE-Seq, RNA-seq and Bisulfite sequecing of mouse germline cells.\n',
 'CaBl: MSL2 is an allelic dosage sensor in mammals\nBiallelic expression is important to ensure adequate mRNA levels as well as protect cells from deleterious heterozygous mutations. However, the factors determining biallelic expression at individual gene loci are unknown. Deletion of the MSL complex component MSL2 in hybrid mouse NPCs uncovers an unprecedented function for MSL2 in allelic regulation. While biallelically expressed in wildtype NPCs, a class of MSL2 targets switches from bi to monoallelic expression upon MSL2 loss.Â\xa0The silenced allele loses nascent transcription, accompanied by dramatic changes in local chromatin landscape such as monoallelic gain of H3K27me3 and DNA methylation. Meanwhile, active histone modifications, RNA POL II, BRD4 and KANSL3 are retained on the active allele. MSL2 targets include several sex-biased genes on autosomes as well as X-chromosomal genes that escape X inactivation. Together, the allele-specific resolution allowed us to unravel an MSL2 mediated safeguarding mechanism to promote biallelic gene expression of sexually dimorphic and dosage-sensitive genes in mammals.\nExpression profiling by high throughput sequencingGenome binding/occupancy profiling by high throughput sequencingMethylation profiling by high throughput sequencing\nAllele specific differential analysis of various sequencing data from ESCs and NPCs of female and male hybrid clones.\n',
 'BlCa: MSL2 is an allelic dosage sensor in mammals\nBiallelic expression is important to ensure adequate mRNA levels as well as protect cells from deleterious heterozygous mutations. However, the factors determining biallelic expression at individual gene loci are unknown. Deletion of the MSL complex component MSL2 in hybrid mouse NPCs uncovers an unprecedented function for MSL2 in allelic regulation. While biallelically expressed in wildtype NPCs, a class of MSL2 targets switches from bi to monoallelic expression upon MSL2 loss.Â\xa0The silenced allele loses nascent transcription, accompanied by dramatic changes in local chromatin landscape such as monoallelic gain of H3K27me3 and DNA methylation. Meanwhile, active histone modifications, RNA POL II, BRD4 and KANSL3 are retained on the active allele. MSL2 targets include several sex-biased genes on autosomes as well as X-chromosomal genes that escape X inactivation. Together, the allele-specific resolution allowed us to unravel an MSL2 mediated safeguarding mechanism to promote biallelic gene expression of sexually dimorphic and dosage-sensitive genes in mammals.\nMus musculus\nExpression profiling by high throughput sequencingGenome binding/occupancy profiling by high throughput sequencingMethylation profiling by high throughput sequencing\nAllele specific differential analysis of various sequencing data from ESCs and NPCs of female and male hybrid clones.\n',
 '9sCa: MSL2 is an allelic dosage sensor in mammals\nBiallelic expression is important to ensure adequate mRNA levels as well as protect cells from deleterious heterozygous mutations. However, the factors determining biallelic expression at individual gene loci are unknown. Deletion of the MSL complex component MSL2 in hybrid mouse NPCs uncovers an unprecedented function for MSL2 in allelic regulation. While biallelically expressed in wildtype NPCs, a class of MSL2 targets switches from bi to monoallelic expression upon MSL2 loss.Â\xa0The silenced allele loses nascent transcription, accompanied by dramatic changes in local chromatin landscape such as monoallelic gain of H3K27me3 and DNA methylation. Meanwhile, active histone modifications, RNA POL II, BRD4 and KANSL3 are retained on the active allele. MSL2 targets include several sex-biased genes on autosomes as well as X-chromosomal genes that escape X inactivation. Together, the allele-specific resolution allowed us to unravel an MSL2 mediated safeguarding mechanism to promote biallelic gene expression of sexually dimorphic and dosage-sensitive genes in mammals.\nMus musculus\nExpression profiling by high throughput sequencingGenome binding/occupancy profiling by high throughput sequencingOtherMethylation profiling by high throughput sequencing\nAllele specific differential analysis of various sequencing data from ESCs and NPCs of female and male hybrid clones.\n',
 'MSL2 is an allelic dosage sensor in mammals\nThis SuperSeries is composed of the SubSeries listed below.\nExpression profiling by high throughput sequencingGenome binding/occupancy profiling by high throughput sequencingMethylation profiling by high throughput sequencingOther\nRefer to individual Series\n',
 'Placenta and fetal brain share a neurodevelopmental disorder DNA methylation profile in a mouse model of prenatal PCB exposure\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nMethylation profiling by high throughput sequencingExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'microRNA-132 regulates adult neural stem cells and their progeny to restore adult hippocampal neurogenesis and memory deficits in Alzheimerâ\x80\x99s disease\nNeural stem cells residing in the hippocampal neurogenic niche sustain life-long neurogenesis in the adult brain. Adult hippocampal neurogenesis (AHN) is functionally linked to mnemonic and cognitive plasticity in humans and rodents. In Alzheimerâ\x80\x99s disease (AD), the process of generating new neurons at the hippocampal neurogenic niche is impeded, yet the mechanisms involved are unknown. Here we identify miR-132, one of the most consistently downregulated microRNAs in AD, as a potent regulator of AHN, exerting cell-autonomous pro-neurogenic effects in the adult neural stem cells and their progeny. Using distinct AD mouse models, cultured human primary and established neural stem cells, and human patient material, we demonstrate that AHN is directly impacted by AD pathology. miR-132 replacement in adult mouse AD hippocampus restores AHN and relevant memory deficits. Our findings corroborate the significance of AHN in AD and reveal the possible therapeutic significance of targeting miR-132 in neurodegeneration.\nMus musculus\nExpression profiling by high throughput sequencing\nNestin:GFP+-niche cells were FACS-sorted from the AD dentate gyrus of Nestin:GFP mice at 2 months of age. Four miR-132-injected and four control-injected Nestin:GFP male mice were used and 94 cells were initially sorted per mouse.\n',
 'Placenta and fetal brain share a neurodevelopmental disorder DNA methylation profile in a mouse model of prenatal PCB exposure [RNA]\nPolychlorinated biphenyls (PCBs) are developmental neurotoxicants implicated as environmental risk factors for neurodevelopmental disorders (NDD). We examined the effects of prenatal exposure to a human-relevant mixture of PCBs on the DNA methylation profiles of fetal mouse brain and placenta. We found thousands of differentially methylated regions (DMRs) distinguishing placentas and brains from PCB-exposed embryos from sex-matched vehicle controls. In both placenta and brain, PCB-associated DMRs were enriched for functions related to neurodevelopment and cellular signaling and enriched within regions of bivalent chromatin. The placenta and brain PCB DMRs overlapped significantly and mapped to a shared subset of genes enriched for Wnt signaling, Slit/Robo signaling, and genes differentially expressed in NDD models. PBC DMRs also significantly overlapped with DMRs from the brains of humans with Rett syndrome and Dup15q syndrome. These results demonstrate that placenta can be used as a surrogate for embryonic brain DNA methylation changes relevant to an NDD.\nMus musculus\nExpression profiling by high throughput sequencing\nPregnant C57BL/6J mice were continuously exposed to 1.0 mg/kg/d mixture of a polychlorinated biphenyls (PCBs). The exposed dams and vehicle matched control dams were sacrificed on gestational day 18 and brain and placenta were harvested from 44 embryos. DNA methylation was assayed by low-pass whole genome bisulfite sequencing (WGBS) and RNA expression was assayed by RNA sequencing (RNA-seq).\n',
 'Placenta and fetal brain share a neurodevelopmental disorder DNA methylation profile in a mouse model of prenatal PCB exposure [WGBS]\nPolychlorinated biphenyls (PCBs) are developmental neurotoxicants implicated as environmental risk factors for neurodevelopmental disorders (NDD). We examined the effects of prenatal exposure to a human-relevant mixture of PCBs on the DNA methylation profiles of fetal mouse brain and placenta. We found thousands of differentially methylated regions (DMRs) distinguishing placentas and brains from PCB-exposed embryos from sex-matched vehicle controls. In both placenta and brain, PCB-associated DMRs were enriched for functions related to neurodevelopment and cellular signaling and enriched within regions of bivalent chromatin. The placenta and brain PCB DMRs overlapped significantly and mapped to a shared subset of genes enriched for Wnt signaling, Slit/Robo signaling, and genes differentially expressed in NDD models. PBC DMRs also significantly overlapped with DMRs from the brains of humans with Rett syndrome and Dup15q syndrome. These results demonstrate that placenta can be used as a surrogate for embryonic brain DNA methylation changes relevant to an NDD.\nMus musculus\nMethylation profiling by high throughput sequencing\nPregnant C57BL/6J mice were continuously exposed to 1.0 mg/kg/d mixture of a polychlorinated biphenyls (PCBs). The exposed dams and vehicle matched control dams were sacrificed on gestational day 18 and brain and placenta were harvested from 44 embryos. DNA methylation was assayed by low-pass whole genome bisulfite sequencing (WGBS).\n',
 'Stromal niche inflammation mediated by IL-1 signaling is a targetable driver of hematopoietic aging\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nExpression profiling by arrayExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'Stromal inflammation is a targetable driver of hematopoietic aging [Microarray analysis]\nWe performed integrated analyses of hematopoietic stem and progenitor cells as well as bone marrow stromal cells to better understand the underlying mechanisms of blood aging.To determine how the molecular state of multipotent progenitor (MPP) cells change with age we performed transcriptomic analyses of myeloid-biased MPP3 and lymphoid-biased MPP4 populations from young and old mice.\nMus musculus\nExpression profiling by array\nMicroarray analysis of 3-5 biological replicated of the indicated MPP populations were isolated from young (8-10 weeks) and old (24 months) wild-type C57BL/6 mice. MPP3s are identified as Lin-/cKit+/Sca-1+/Flk2-/CD48+/CD150-, and MPP4s are identified as Lin-/cKit+/Sca-1+/Flk2+. Young controls listed here are also available under GSE68529.\n',
 'Inflammation of the Aged Bone Marrow Niche Drives Blood Aging\nHematopoietic aging is defined by a loss of regenerative capacity and skewed differentiation from hematopoietic stem cells (HSC) leading to dysfunctional blood production. Signals from the bone marrow (BM) microenvironment dynamically tailor hematopoiesis, but the effect of aging on the niche and the contribution of the aging niche to blood aging still remains unclear. Here, we show the development of an inflammatory milieu in the aged marrow cavity, which drives both niche and hematopoietic system remodeling. We find decreased numbers and functionality of osteogenic endosteal mesenchymal stromal cells (MSC), expansion of pro-inflammatory perisinusoidal MSCs, and deterioration of the central marrow sinusoidal endothelium, which together create a self-reinforcing inflamed BM milieu. Single cell molecular mapping of old niche cells further confirms disruption of cell identities and enrichment of inflammatory response genes. Inflammation, in turn, drives chronic activation of emergency myelopoiesis pathways in old HSCs and multipotent progenitors, which promotes myeloid differentiation at the expense of lymphoid and erythroid commitment, and hinders hematopoietic regeneration. Remarkably, both defective hematopoietic regeneration, niche deterioration and HSC aging can be improved by blocking inflammatory IL-1 signaling. Our results indicate that targeting the pro-inflammatory niche milieu can be instrumental in restoring blood production during aging.\nMus musculus\nExpression profiling by high throughput sequencing\nRNA sequencing of mesenchymal and endothelial populations isolated from both endosteum and central marrow of young and old mice\n',
 'Stromal inflammation is a targetable driver of hematopoietic aging [droplet-based scRNAseq]\nTo investigate global changes in the structure of the bone marrow niche with age, we performed droplet-based scRNAseq (10X Genomics) of unfractionated endosteal and central marrow stromal cells (Ter119-/CD45-). 8,735 cells passed QC, and hematopoietic clusters of cells were excluded based on marker gene expression, leaving 2359 cells distributed across 9 distinct clusters for final presentation.\nMus musculus\nExpression profiling by high throughput sequencing\nEndosteal and central marrow stromal cells from the bone marrow of young (10 weeks) and old (24 months) wild-type C57BL/6 mice were isolated by flow cytometry and subjected to droplet-based scRNAseq (10X Genomics).\n',
 'Dementia subtype prediction models constructed by penalized regression methods for multiclass classification using serum microRNA expression data\nThere are many subtypes of dementia, and identification of diagnostic biomarkers that are minimally-invasive, low-cost, and efficient is desired. Circulating microRNAs (miRNAs) have recently gained attention as easily accessible and non-invasive biomarkers. We conducted a comprehensive miRNA expression analysis of serum samples from 1348 Japanese dementia patients, composed of four subtypesâ\x80\x94Alzheimerâ\x80\x99s disease (AD), vascular dementia, dementia with Lewy bodies (DLB), and normal pressure hydrocephalusâ\x80\x94and 246 control subjects. We used this data to construct dementia subtype prediction models based on penalized regression models with the multiclass classification. We constructed a final prediction model using 46 miRNAs, which classified dementia patients from an independent validation set into four subtypes of dementia. Network analysis of miRNA target genes revealed important hub genes, SRC and CHD3, associated with the AD pathogenesis. Moreover, MCU and CASP3, which are known to be associated with DLB pathogenesis, were identified from our DLB-specific target genes. Our study demonstrates the potential of blood-based biomarkers for use in dementia-subtype prediction models. We believe that further investigation using larger sample sizes will contribute to the accurate classification of subtypes of dementia.\nHomo sapiens\nExpression profiling by array\nSerum samples from 84 patients of normal pressure hydrocephalus (NPH)\n',
 "Bulk RNA-seq of aplastic anemia (AA) and healthy donors\nBulk RNA-seq data of  Lin-CD34+ hematopoietic stem and progenitor cells derived from bone marrow of healthy donors and untreated aplastic anemia  patients\nHomo sapiens\nExpression profiling by high throughput sequencing\nMononuclear cells from 3 healthy donors and untreated aplastic anemia patients' bone marrow aspirates were isolated using Ficoll density gradient separation and cryopreserved in 90% FBS/ 10% DMSO for storage in liquid nitrogen. The thawing frozen cells were stained with human antibodies including lineage cocktail (CD3, CD14, CD16, CD19, CD20, CD56) (BV510, Biolegend) and CD34 (APC, Clone 581, BD Biosciences) antibodies. Lin-CD34+ HSPCs (cell count ranges from 130 to 2000) were sorted directly into cell lysis buffer on a BD FACSAria III. The reverse transcription and template switch steps were performed following the full-length Smart-seq2 protocol with some modifications. Concentration of oligd(T30VN) and template switch oligo (TSO) primers used for mRNA reverse transcription were double. The cDNA was fragmented using Covaris S220 instead of Tn5 transposase. The processes of end repair, polyA-tailing, adaptor ligation and library amplification were using KAPA Hyper Prep Kits (Kapa Biosystems, Cat# KK8504) and NEBNext Multiplex Oligos for Illumina (Index Primers Set 1) (NEB, Cat# E7335L).The 150bp paired-end reads were generated from Illumina NovaSeq6000 platform. For data processing, TSO primer, polyA tail, adaptor, and low-quality sequence (N > 10%) were trimmed using Trimmomatic (version 0.36). Clean reads were aligned against the human GRCh38 genome using STAR (version 2.7)). Duplicate reads were identified and removed by using Picard(version 2.17.6). MATS(version 4.0.2) and DaPars(version 0.9.0) to perform alternative splicing and alternative polyadenylation analysis, respectively. Gene quantification was using HTSeq (version 0.9.1).\n",
 "Machine Learning Identifies Candidates for Drug Repurposing in Alzheimer's Disease\nClinical trials of novel therapeutics for Alzheimer's Disease (AD) have consumed a large amount of time and resources with largely negative results. Repurposing drugs already approved by the Food and Drug Administration (FDA) for another indication is a more rapid and less expensive option. In this study, we profile 80 FDA-approved and clinically tested drugs in neural cell cultures, with the goal of producing a ranked list of possible repurposing candidates.\nHomo sapiens\nExpression profiling by high throughput sequencing\nAn in-vitro differentiated mixture of neuron and glial cells derived from the ReNcell VM neural progenitor cell line was treated with 80 different compounds and mRNA levels measured using RNA-seq.\n",
 'A novel combination of serum microRNAs for the detection of early gastric cancer\nA serum miRNA combination could be a powerful classifier for the detection of patients with early stage gastric cancer.\nHomo sapiens\nNon-coding RNA profiling by array\nSerum microRNA profiles of 2940 samples, which consist of 1423 gastric cancers, 1417 non-cancer controls, 50 esophageal cancers, and 50 colorectal cancers.\n',
 'Stat1 activation, through systemic interferon signaling, and upregulation of innate immune system receptors drives inflammaging, that is ameliorated by dietary restriction intervention\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencingExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'Stat1 activation, through systemic interferon signaling, and upregulation of innate immune system receptors drives inflammaging, that is ameliorated by dietary restriction intervention [RNA-seq]\nAging is the result of an accumulation of molecular damages, driven by multiple factors like metabolic and oxidative stress, (epi)genetic alterations, and microbiome dysbiosis. In addition, aging is characterized by a chronic, low-grade inflammation known as inflammaging that represents a high significant risk factor for most, if not all, age-related diseases. Dietary restriction (DR) is the best-known non-invasive method to ameliorate the aging phenotype in many organisms. While it has been shown that DR reduces the inflammation in some tissues, the molecular mechanisms in old individuals are still largely unknown. Here we identify a multi-tissue gene network which regulates the inflammaging and that is ameliorated by DR. Using transcriptional profiling (RNA-seq), we found that aging upregulates the innate immune system receptor together with the activation of a systemic interferon (IFN) signaling through the Interferon Regulatory Factors (Irf), inflammatory cytokines and finally Stat1 transcription factor. Our results suggest that DR is able to ameliorate inflammaging by downregulating the expression and the activity of almost all the components of this gene network. In addition, chromatin accessibility genome-wide analysis showed that the binding site of Irf and Stat1 in the promoter of their target genes are more open in aging and become closer upon DR treatment indicating a transcriptional control of the inflammaging phenotype involving epigenetic modifications. Our results provide a comprehensive understanding of the molecular network regulating the inflammation in aging and DR and present novel anti-inflammaging therapeutic targets.\nMus musculus\nExpression profiling by high throughput sequencing\nTranscriptional profile of kidney, liver, lung, muscle, skin, brain, blood and heart in aging, short-term and long-term dietary restriction.\n',
 'Stat1 activation, through systemic interferon signaling, and upregulation of innate immune system receptors drives inflammaging, that is ameliorated by dietary restriction intervention [ATAC-seq]\nAging is the result of an accumulation of molecular damages, driven by multiple factors like metabolic and oxidative stress, (epi)genetic alterations, and microbiome dysbiosis. In addition, aging is characterized by a chronic, low-grade inflammation known as inflammaging that represents a high significant risk factor for most, if not all, age-related diseases. Dietary restriction (DR) is the best-known non-invasive method to ameliorate the aging phenotype in many organisms. While it has been shown that DR reduces the inflammation in some tissues, the molecular mechanisms in old individuals are still largely unknown. Here we identify a multi-tissue gene network which regulates the inflammaging and that is ameliorated by DR. Using transcriptional profiling (RNA-seq), we found that aging upregulates the innate immune system receptor together with the activation of a systemic interferon (IFN) signaling through the Interferon Regulatory Factors (Irf), inflammatory cytokines and finally Stat1 transcription factor. Our results suggest that DR is able to ameliorate inflammaging by downregulating the expression and the activity of almost all the components of this gene network. In addition, chromatin accessibility genome-wide analysis showed that the binding site of Irf and Stat1 in the promoter of their target genes are more open in aging and become closer upon DR treatment indicating a transcriptional control of the inflammaging phenotype involving epigenetic modifications. Our results provide a comprehensive understanding of the molecular network regulating the inflammation in aging and DR and present novel anti-inflammaging therapeutic targets.\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencing\nATAC-seq (Assay for Transposase-Accessible Chromatin using sequencing) of kidney and liver in young, old, and long-term dietary restriction.\n',
 'Age-related changes in blood and spleen-derived immune cells in long and short lived mice strains\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencingExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 "Age-related changes in blood and spleen-derived immune cells in long and short lived mice strains [RNA-seq]\nImmune system aging contributes significantly to declining health with age. It is unknown to what extent human and mice immune systems go through similar age-related changes and whether there are conserved biomarkers of aging. We characterized age-related changes in the immune system of long (C57BL/6J) and short-lived (NZO/HILtJ) strains and compared them with blood-driven human aging signatures. Peripheral blood lymphocytes (PBL), spleen cells and spleen-driven naive and memory CD8+ T cells were profiled using flow cytometry, RNA-seq and ATAC-seq from young (3 months) and old (18 months) mice. Pro-inflammatory myeloid genes were activated with age across strains and tissues, echoing human 'inflammaging' signatures. ATAC-seq footprinting analyses uncovered increased binding of pro-inflammatory transcription factors with age (e.g., AP1 complex, NFKB signaling pathway). Machine learning models identified inflammation, cytotoxic, and naive T cell derived genes to be strong signatures of immunosenescence. These data are publicly shared as a resource for immune aging (https://immune-aging.jax.org/mice).\nMus musculus\nExpression profiling by high throughput sequencing\nSpleen, PBL and spleen sorted CD8+ T cell profiles of B6 and NZO mice at 3,12 and 18 months\n",
 "Age-related changes in blood and spleen-derived immune cells in long and short lived mice strains [ATAC-seq]\nImmune system aging contributes significantly to declining health with age. It is unknown to what extent human and mice immune systems go through similar age-related changes and whether there are conserved biomarkers of aging. We characterized age-related changes in the immune system of long (C57BL/6J) and short-lived (NZO/HILtJ) strains and compared them with blood-driven human aging signatures. Peripheral blood lymphocytes (PBL), spleen cells and spleen-driven naive and memory CD8+ T cells were profiled using flow cytometry, RNA-seq and ATAC-seq from young (3 months) and old (18 months) mice. Pro-inflammatory myeloid genes were activated with age across strains and tissues, echoing human 'inflammaging' signatures. ATAC-seq footprinting analyses uncovered increased binding of pro-inflammatory transcription factors with age (e.g., AP1 complex, NFKB signaling pathway). Machine learning models identified inflammation, cytotoxic, and naive T cell derived genes to be strong signatures of immunosenescence. These data are publicly shared as a resource for immune aging (https://immune-aging.jax.org/mice).\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencing\nSpleen, PBL and spleen sorted CD8+ T cell profiles of B6 and NZO mice at 3,12 and 18 months\n",
 'Non-coding RNAs shape cortical neurons developmental trajectories\nA hallmark of cortical evolution is the high dynamic subventricular zone (SVZ) expansion, where basal progenitors (BPs) amplify and neuronal transcriptional programs unfold. How non-coding molecular factors such as microRNAs influence these developmental trajectories and regulate the acquisition of cortical type identities is largely unknown. Here we demonstrate that miR-137 and miR-122 regulate the positioning and identity features of superficial layer cortical neurons by acting at distinct steps of their developmental trajectories. MiR-137 sustains basal progenitor amplification by reverting their neurogenic commitment and inducing high proliferative state upregulating Cd63 and inhibiting Myt1l. Cd63 is an extra-cellular matrix (ECM) receptor which interacts with ï\x81¡ï\x81®b3- and ï\x81¢1-integrin pathways to promote proliferation, while Myt1l is a transcription factor that promotes and sustains neuronal fate. The BPs amplification by miR-137 is converted in the promotion of intracortical projecting neuron (ICPN) identity and L2/3 expansion. As opposed to miR-137, miR-122 acts postmitotically, affecting the bioelectrical properties, the calcium and cytoskeleton dynamics of newborn neurons as well as their transcriptional program, leading to a persistent molecular immaturity across time. Overall, these findings reveal that miR-137 and miR-122 are key regulators of the developmental trajectory of cortical neurons across evolution.\nMus musculus\nExpression profiling by high throughput sequencing\nInvestigation of the non-coding RNAs miR-122 and miR-137 overexpression on progenitor and neuron transcriptional programs.   psil-miR122, psil-miR137 or psil-scramble (control) plasmids were in utero electroporated together with pUB-GFP or pUB-Tomato plasmid in mouse embryos at embryonic day (E) 14.5, and cells collected at E15.5, E17.5 or postnatal day (P) 7 using fluorescence activated cell sorting (FACS).   For comparison with layer 4 and layer 2/3 control cell molecular identities, pUB-GFP/Tomato plasmid was in utero electroporated at E14 and E15.5 respectively and cells collected at P3 and P7 using FACS.   Collected cells were then processed for single-cell transcriptomics.  psil-miR122 was in utero electroporated together with pUB-GFP plasmid in mouse embryos at E14.5; superficial and deep layers were microdissected at P7 and cells collected using FACS before being processed for bulk RNA sequencing.\n',
 "Transcriptional and imprinting complexity in Arabidopsis seeds at single-nucleus resolution\nSeeds are the basis of agriculture, yet their full transcriptional complexity has remained unknown. Here, we employ single-nucleus RNA-sequencing to characterize developing Arabidopsis thaliana seeds, with a focus on endosperm. Endosperm, the site of gene imprinting in plants, mediates the relationship between the maternal parent and embryo. We identify new cell types in the chalazal endosperm region, which interfaces with maternal tissue for nutrient unloading. We further demonstrate that the extent of parental bias of maternally expressed imprinted genes varies with cell cycle phase, and that imprinting of paternally expressed imprinted genes is strongest in chalazal endosperm. These data indicate imprinting in endosperm is heterogeneous and suggest that parental conflict, which is proposed to drive the evolution of imprinting, is fiercest at the boundary between filial and maternal tissues.\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nTo identify cell/nuclei types and investigate imprinting dynamics within endosperm, single nucleus RNA-seq was performed using the Smart-seq2 method on seed nuclei, targeting either the 3C or 6C FANS peak in order to enrich for triploid endosperm (other seed tissues are diploid). A total of 1,664 libraries were sequenced, of which 64 were negative controls (no DNA in library prep or no cell sorted), 51 were made from two nuclei to test single-nuclei sorting accuracy, 112 failed QC checks (< 1,500 genes detected and/or <1,000 detected with 5+ reads), and the remaining 1,437 were high-quality single-nuclei libraries used in final analysis. The final dataset consists primarily of seeds derived from Col, Cvi reciprocal crosses at 4 days after pollination (DAP), although some nuclei were instead obtained from Col, Ler crosses and/or other timepoints.Please note that the counts.txt files for the P17_10A, P21_10A, and P22_10A samples are identical as they are from negative controls that should not have any reads mapping to A. thaliana.There are  ~50 negative controls in the dataset, and most have at least one or two reads falling in a gene somewhere and so the processed data files (read counts per gene) aren't exactly the same, but the read counts in all three files are actually zero for all genes, so all three files are identical.\n",
 'RNA-seq of Arabidopsis thaliana seedlings exposed to different communities of commensal bacteria [SSC]\nThis study evaluates the transcriptome of Arabidopsis thaliana seedlings exposed to communities of suppressors (SynCom5-S1) and non-suppressors (SynCom5-NS1) of the plant immune system\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nArabidopsis seeds (wild-type Col-0) were germinated on Johnson medium and grown for 7 days in the vertical position. Plants were then transferred to media containing SynCom5-S1, SynCom5-NS1 (10E5 c.f.u / mL per strain), or no bacteria. Plants were grown in a growth chamber in a 15-h dark/9-h light regime (21Â°C day /18Â°C night). Seedlings were harvested twelve days after treatment. The experiment includes six biological replicates from two independent experiments.\n',
 'RNA-seq of Arabidopsis thaliana roots exposed to the MAMP flg22 in the presence of commensal bacteria\nThis SuperSeries is composed of the SubSeries listed below.\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'RNA-seq of Arabidopsis thaliana roots exposed to the MAMP flg22 in the presence of a community of commensal bacteria [RST]\nThis study evaluates the transcriptome of Arabidopsis thaliana roots exposed to the MAMP flg22 in the presence of a 35-member bacterial SynCom\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nArabidopsis seeds (wild-type Col-0) were germinated on Johnson medium and grown for 7 days in the vertical position. Plants were then transferred to media containing 1 uM flg22 or not in the presence of SynCom35 (10E5 c.f.u / mL per strain), heat-killed SynCom35 or no bacteria. Plants were grown in a growth chamber in a 15-h dark/9-h light regime (21Â°C day /18Â°C night). Roots were harvested one or twelve days after treatment. The experiment includes nine biological replicates from three independent experiments.\n',
 'RNA-seq of Arabidopsis thaliana seedlings exposed to the MAMP flg22 in the presence of individual commensal bacteria [MAE]\nThis study evaluates the transcriptome of Arabidopsis thaliana seedlings exposed to the MAMP flg22 in the presence of 35 different bacteria\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nArabidopsis seeds (wild-type Col-0) were germinated on Johnson medium and grown for 7 days in the vertical position. Plants were then transferred to media containing 1 uM flg22 or not in the presence of a commensal bacteria strain (10E5 c.f.u / mL). Plants were grown in a growth chamber in a 15-h dark/9-h light regime (21Â°C day /18Â°C night). Whole seedlings were harvested twelve days after treatment. The experiment includes nine biological replicates from three independent experiments.\n',
 'Effects of vitamin D on non-specific inflammatory response in aged skin.\nWe have previously demonstrated older skin exhibits increased sterile inflammation 6 hours after saline-injection. In this work, we examined whether vitamin D in vivo would attenuate this non-specific inflammatory response towards saline in elder individuals (â\x89¥65 years).\nHomo sapiens\nExpression profiling by high throughput sequencing\nSkin mRNA profiles 6 hours after saline injection were studied before and after vitamin D supplementation.\n',
 'Altered microRNA expression links IL6 and TNF-induced inflammaging with myeloid malignancy\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nMethylation profiling by high throughput sequencingExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'Differential gene expression in miR-146a-/- vs. WT hematopoietic stem and progenitor cells [miR146aKO_vs_WT_RNA-seq_bulk]\nTo identify pathways and processes driving the observed hematopoietic stem cell (HSC) aging-like phenotypes in miR-146a-/- vs. WT, we performed RNA-seq gene expression profiling of Lin- Sca-1+ c-Kit+ (LSK) cells isolated from miR-146a-/- or WT mouse bone marrow (BM). Differential expression analysis and EnrichmentMap network analysis identified cytokine signalling and immune pathways as potential drivers of aging-like alterations in miR-146a-/- HSC proliferation and differentiation.\nMus musculus\nExpression profiling by high throughput sequencing\nRNA sequencing of miR-146a-/- vs. WT (C57BL/6J) Lin- Sca-1+ c-Kit+ cells from young mice.\n',
 'Altered microRNA expression links IL6 and TNF-induced inflammaging with myeloid malignancy [miR146aKO_LSK_WGBS]\nAging is associated with significant changes in the hematopoietic system, including increased inflammation, impaired hematopoietic stem cell (HSC) function, and increased incidence of myeloid malignancy. Inflammation of aging (â\x80\x9cinflammagingâ\x80\x9d) has been proposed as a driver of age-related changes in HSC function and myeloid malignancy, but mechanisms linking these phenomena remain poorly defined. Here, we identify loss of miR-146a as driving aging-associated inflammation in AML patients. miR-146a expression declined in old wild-type mice, and loss of miR-146a promoted premature HSC aging and inflammation in young miR-146a-null mice, preceding development of aging-associated myeloid malignancy. Using single-cell assays of HSC quiescence, stemness, differentiation potential, and epigenetic state to probe HSC function and population structure, we found that loss of miR-146a depleted a subpopulation of primitive, quiescent HSCs. DNA methylation and transcriptome profiling implicated NF-ÎºB, IL6, and TNF as potential drivers of HSC dysfunction, activating an inflammatory signaling relay promoting IL6 and TNF secretion from mature miR-146a-/- myeloid and lymphoid cells. Reducing inflammation by targeting Il6 or Tnf was sufficient to restore single-cell measures of miR-146a-/- HSC function and subpopulation structure, and reduced the incidence of hematological malignancy in miR 146a-/- mice. miR-146a-/- HSCs exhibited enhanced sensitivity to IL6 stimulation, indicating that loss of miR-146a affects HSC function via both cell-extrinsic inflammatory signals and increased cell-intrinsic sensitivity to inflammation. Thus, loss of miR 146a regulates cell-extrinsic and -intrinsic mechanisms linking HSC inflammaging to the development of myeloid malignancy.\nMus musculus\nMethylation profiling by high throughput sequencing\n2 genotypes, 1 cell type: WT LSK (see GSE95697), miR-146a-/- LSK (this series)\n',
 'Altered microRNA expression links IL6 and TNF-induced inflammaging with myeloid malignancy [miR146aKO_ESLAM_SCBS]\nAging is associated with significant changes in the hematopoietic system, including increased inflammation, impaired hematopoietic stem cell (HSC) function, and increased incidence of myeloid malignancy. Inflammation of aging (â\x80\x9cinflammagingâ\x80\x9d) has been proposed as a driver of age-related changes in HSC function and myeloid malignancy, but mechanisms linking these phenomena remain poorly defined. Here, we identify loss of miR-146a as driving aging-associated inflammation in AML patients. miR-146a expression declined in old wild-type mice, and loss of miR-146a promoted premature HSC aging and inflammation in young miR-146a-null mice, preceding development of aging-associated myeloid malignancy. Using single-cell assays of HSC quiescence, stemness, differentiation potential, and epigenetic state to probe HSC function and population structure, we found that loss of miR-146a depleted a subpopulation of primitive, quiescent HSCs. DNA methylation and transcriptome profiling implicated NF-ÎºB, IL6, and TNF as potential drivers of HSC dysfunction, activating an inflammatory signaling relay promoting IL6 and TNF secretion from mature miR-146a-/- myeloid and lymphoid cells. Reducing inflammation by targeting Il6 or Tnf was sufficient to restore single-cell measures of miR-146a-/- HSC function and subpopulation structure, and reduced the incidence of hematological malignancy in miR 146a-/- mice. miR-146a-/- HSCs exhibited enhanced sensitivity to IL6 stimulation, indicating that loss of miR-146a affects HSC function via both cell-extrinsic inflammatory signals and increased cell-intrinsic sensitivity to inflammation. Thus, loss of miR 146a regulates cell-extrinsic and -intrinsic mechanisms linking HSC inflammaging to the development of myeloid malignancy.\nMus musculus\nMethylation profiling by high throughput sequencing\n2 cell types, 2 genotypes, single-cell analysis: 64 WT LSK cells (see GSE89545), 83 WT ESLAM cells (see GSE89545), & 69 miR-146a-/- ESLAM cells, 43 passing QC (this series)\n',
 'Single cell RNA-seq of psoriatic skin identifies pathogenic Tc17 subsets and reveals distinctions between CD8+ T cells in autoimmunity and cancer\nPsoriasis is an inflammatory, IL-17-driven skin disease in which autoantigen-induced CD8+ T cells have been identified as pathogenic drivers. In this study, we used single-cell RNA-seq to identify 11 transcriptionally diverse CD8+ T cell subsets in psoriatic and healthy skin. Among several inflammatory subsets enriched in psoriatic skin, we observed two Tc17 subsets that were metabolically divergent, developmentally related, and expressed CXCL13, which we found to be a biomarker of psoriasis severity. Despite high co-inhibitory receptor expression in the Tc17 clusters, a comparison of these cells with melanoma-infiltrating CD8+ T cells revealed upregulated cytokine, cytolytic, and metabolic transcriptional activity in the psoriatic cells that differed from an exhaustion program. Our findings provide a high-resolution view of cutaneous CD8+ T cells in psoriasis and healthy skin and shed light on their functional distinctions within the chronic pathologies of autoimmunity and cancer.\nHomo sapiens\nExpression profiling by high throughput sequencing\ntranscriptomic comparison of CD8+ T cells in 11 psoriatic and 5 healthy skin samples\n',
 'single cell RNA-seq analysis of ovarian cancer ascites\nTo understand the cellular diversity within malignant ascites, we profiled ~11,000 single cells from 22 samples by Smartseq2 and by 10x.\nHomo sapiens\nExpression profiling by high throughput sequencing\nAscites samples were either depleted of CD45+ cells and profiled by 10x or sorted by EPCAM+CD24+ and profiled by SmartSeq2Note from Submitter: We have difficulties in approving raw data submission of data from clinical patient samples to GEO or dbGAP and are therefore submitting at this point only the processed data and the metadata.  In parallel with the process of granting approval for raw data submission to dbGAP we are also using another repository to which we are allowed to deposit raw data (DUOS@broadinstitute.org).\n',
 'Transcriptome analysis of HSPCs and T cells from aplastic anemia (AA) and healthy donors\nThis SuperSeries is composed of the SubSeries listed below.\nHomo sapiens\nExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 "Single cell transcriptome from aplastic anemia (AA) and healthy donors [3' RNA-seq]\nTranscriptome analysis of hematopoietic stem and progenitor cells (HSPCs) and T cells collected from bone marrow and peripheral blood of healthy donors and aplastic anemia patients untreated or response to immunosuppressive therapy.\nHomo sapiens\nExpression profiling by high throughput sequencing\n3' end mRNA profiles of hematopoietic stem and progenitor cells (HSPCs) and T cells under aplastic anemia (AA) and healthy conditions were generated by deep sequencing at single cell resolution, using Illumina Hiseq\n",
 'Single cell transcriptome from aplastic anemia (AA) and healthy donors [Full-length RNA-seq]\nTranscriptome analysis of hematopoietic stem and progenitor cells (HSPCs) from aplastic anemia (AA) and healthy donors\nHomo sapiens\nExpression profiling by high throughput sequencing\nFull length mRNA profiles of hematopoietic stem and progenitor cells (HSPCs) under aplastic anemia (AA) and healthy conditions were generated by deep sequencing at single cell resolution, using Illumina Hiseq\n',
 'Cellular trajectory analysis links tissue maturation to cellular specialization in the plant meristem [protophloem sieve element cells]\nThe mechanisms that allow cells in the plant meristem to coordinate tissue-wide maturation gradients with specialized cell networks are critical for indeterminate growth. Here, we combine long-term imaging, single-cell transcriptomics, genetic analysis, and network modeling to reconstruct the differentiation trajectory of the 20 protophloem cells in the Arabidopsis root. We found that lineage bifurcation and cellular specification are mediated near the stem cell niche by PEAR regulators. However, many PEAR downstream effects are repressed by a broad gradient of PLETHORA transcription factors, which directly inhibit PEARsâ\x80\x99 own direct target APL until dissipating seven cells from the stem cell niche. Modeling and molecular epistasis suggest a so-called â\x80\x9cseesawâ\x80\x9d cascade of sharp transitions in which transcription factors regulating consecutive steps mutually inhibit each otherâ\x80\x99s targets, ending in programmed enucleation. This analysis provides a mechanistic understanding of how morphogen-like maturation gradients interface with cell-type specific transcriptional regulators to stage cellular differentiation\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nRNA seq profiling of bulk-isolated (FACS) protophloem sieve element cells\n',
 'Cellular trajectory analysis links tissue maturation to cellular specialization in the plant meristem II\nThe mechanisms that allow cells in the plant meristem to coordinate tissue-wide maturation gradients with specialized cell networks are critical for indeterminate growth. Here, we combine long-term imaging, single-cell transcriptomics, genetic analysis, and network modeling to reconstruct the differentiation trajectory of the 20 protophloem cells in the Arabidopsis root. We found that lineage bifurcation and cellular specification are mediated near the stem cell niche by PEAR regulators. However, many PEAR downstream effects are repressed by a broad gradient of PLETHORA transcription factors, which directly inhibit PEARsâ\x80\x99 own direct target APL until dissipating seven cells from the stem cell niche. Modeling and molecular epistasis suggest a so-called â\x80\x9cseesawâ\x80\x9d cascade of sharp transitions in which transcription factors regulating consecutive steps mutually inhibit each otherâ\x80\x99s targets, ending in programmed enucleation. This analysis provides a mechanistic understanding of how morphogen-like maturation gradients interface with cell-type specific transcriptional regulators to stage cellular differentiation\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nRNA seq profiling of mutant and overexpressor lines\n',
 'Cellular trajectory analysis links tissue maturation to cellular specialization in the plant meristem\nThe mechanisms that allow cells in the plant meristem to coordinate tissue-wide maturation gradients with specialized cell networks are critical for indeterminate growth. Here, we combine long-term imaging, single-cell transcriptomics, genetic analysis, and network modeling to reconstruct the differentiation trajectory of the 20 protophloem cells in the Arabidopsis root. We found that lineage bifurcation and cellular specification are mediated near the stem cell niche by PEAR regulators. However, many PEAR downstream effects are repressed by a broad gradient of PLETHORA transcription factors, which directly inhibit PEARsâ\x80\x99 own direct target APL until dissipating seven cells from the stem cell niche. Modeling and molecular epistasis suggest a so-called â\x80\x9cseesawâ\x80\x9d cascade of sharp transitions in which transcription factors regulating consecutive steps mutually inhibit each otherâ\x80\x99s targets, ending in programmed enucleation. This analysis provides a mechanistic understanding of how morphogen-like maturation gradients interface with cell-type specific transcriptional regulators to stage cellular differentiation\nExpression profiling by high throughput sequencing\nIsolation and RNA seq of developing phloem cells in the root meristem of Arabidopsis thaliana\n',
 'Multi-omic profiling of single nuclei uncovers regulatory diversity of brain cell types and diseases\nMost existing single-cell techniques can only make one type of molecular measurements. Although computational approaches have been developed to integrate single-cell datasets, their efficacy still needs to be determined with reference to authentic single-cell multi-omic profiles. To address this challenge, we devised single-nucleus methylCytosine, Chromatin accessibility and Transcriptome sequencing (snmC2T-seq) and applied the approach to post-mortem human frontal cortex tissue. We developed a computational framework to evaluate the quality of finely defined cell types using multi-modal information and validated the efficacy of computational multi-omic integration methods. Correlation analysis in individual cells revealed gene groups showing distinct relations between methylation and expression. Integration of snmC2T-seq with other multi- and single- modal datasets enabled joint analyses of the methylome, chromatin accessibility, transcriptome, and chromatin architecture for 63 human cortical cell types. We reconstructed the regulatory lineage of these cortical cell types and found pronounced cell-type-specific enrichment of disease risks for neuropsychiatric traits, predicting causal cell types that can be targeted for treatment.\nHomo sapiens\nGenome binding/occupancy profiling by high throughput sequencingMethylation profiling by high throughput sequencingExpression profiling by high throughput sequencing\nsnmC2T-seq and snATAC-seq\n',
 'Stem cell derived human microglia transplanted in mouse brain to study human disease\nWhile genetics highlight the role of microglia in Alzheimerâ\x80\x99s disease (AD), one third of putative AD-risk genes lack adequate mouse orthologs. Here, we successfully engraft human microglia derived from embryonic stem cells in the mouse brain. The cells recapitulate transcriptionally human primary microglia ex vivo and show expression of human specific AD-risk genes. Oligomeric Amyloid-Î² induces a divergent response in human vs. mouse microglia. This model can be used to study the role of microglia in neurological diseases.\nExpression profiling by high throughput sequencing\nTo control for variability across different experiments, tissue from three mice was pooled in all experimental conditions, from different litters and transplanted with H9-microglia produced in independent batches of in vitro differentiation. Across the manuscript we report both the number of mice/differentiations pooled, as well as the number of sequencing pools. In summary, 1) we sequenced 2,246 transplanted H9-microglia (n=3/1, 3 mice in 1 combined sequencing pool), 4496 H9-derived monocytes (n=2/1, 2 differentiations in 1 combined sequencing pool) and 3385 microglia in vitro (n=2/1), and 22,846 human primary microglia obtained from cortical surgical resections. 2) We sequenced 4880 transplanted H9-microglia (n=3x2/2) and 9942 host mouse cells (n=3x2/2) and assessed their differential response to oligomeric amyloid beta.\n',
 'Exploring the effects of rearing densities on epigenetic modifications in the zebrafish gonads\nThe goal of this study was to determine whether there was a direct relationship between stress and DNA methylation changes in fish gonads.  During sex differentiation, we studied the sensitive window to density. Sex ratios were studied and methylation of canonical reproduction-related genes was analysed.\nDanio rerio\nMethylation profiling by high throughput sequencing\nZebrafish were reared at high density conditions as described in Ribas et al., 2017 J. Exp. Biol. During 7-21 days post fertilization (dpf) and 18-45 dpf. At 90 dpf gonads were dissected and flash frozen with liquid nitrogen for further molecular analysis and sex ratios were recorded\n',
 'Global transcriptomic profiling of the bone marrow stromal microenvironment during postnatal development, aging, and inflammation\nRNA-sequencing of a longitudinal comparison of four principal mesenchymal and endothelial stromal cell types (CXCL12-abundant reticular cells, PDGFR-Î±+ Sca-1+, sinusoidal and arterial endothelial cells), isolated from early postnatal, adult and aged mice. Additional transcriptional profiling of the response of CXCL12-abundant reticular cells and sinusoidal endothelial cells to infection-mimicking agents.\nMus musculus\nExpression profiling by high throughput sequencing\nRNA-sequencing of four FACS-isolated mouse bone marrow stromal cells at different age stages and conditions.\n',
 'Simultaneous profiling of 3D genome structure and DNA methylation in single human cells\nDynamic 3D chromatin conformation is a critical mechanism for gene regulation during development and disease. Despite this, profiling of 3D genome structure from complex tissues with cell-type specific resolution remains challenging. Recent efforts have demonstrated that cell-type specific epigenomic features can be resolved in complex tissues using single-cell assays. However, it remains unclear whether single-cellChromatin Conformation Capture (3C) or Hi-C profiles can effectively identify cell types and reconstruct cell-type specific chromatin conformation maps. To address these challenges, we have developed single-nucleus methyl-3C sequencing (sn-m3C-seq) to capture chromatin organization and DNA methylation information and robustlyseparate heterogeneous cell types. Applying this method to >4,200 single human brain prefrontal cortex cells, we reconstruct cell-type specific chromatin conformation maps from 14 cortical cell types. These datasets reveal the genome-wide association between cell-type specific chromatin conformation and differential DNA methylation, suggestingpervasive interactions between epigenetic processes regulating gene expression.\nHomo sapiens\nMethylation profiling by high throughput sequencingOther\nMulti-omic method single-nucleus methyl-3C sequencing (sn-m3C) to profile chromatin conformation and DNA methylation from the same cell.\n',
 'Effects of losmapimod on antigen-specific response in aged skin\nOlder humans are more susceptible to infections, even those to which they were immune to in their youth. Vaccination against many different infectious agents is also sub-optimal in these individuals. To investigate the mechanisms involved in the decline in immunity during ageing we inject safe derivatives of microorganisms into the skin of old and young volunteers.  After injection of the agent, we can extract the white cells (leucocytes) that accumulate at the site of the immune response to test for their function.  One of the responses we have been investigating is that to the varicella zoster virus (VZV) that induces chickenpox mainly in young individuals. Infected subjects control this persistent virus until an older age where reduced immunity leads to viral re-activation to cause shingles.  We have found that this defective response is associated with increased baseline inflammation in this tissue. Although inflammatory responses are required to clear infections, excessive inflammation has previously been shown to interfere with the generation of specific immunity.  We will therefore investigate ways of enhancing the immune response of older humans by blocking basal inflammation.     To extend our initial observations of high background inflammation in the skin, we will investigate which cells in this tissue are responsible for the production of inflammatory mediators by using a second human experimental system that tests the response to a skin irritant known as cantharadin. Next we will block inflammation in the skin to determine whether this can lead to improved responses upon challenge with antigens. This will be achieved by pre-treating old subjects with a drug that has already been developed by GSK (Losmapimod, p38MAPkinase inhibitor) and another that is available off the shelf (vitamin D3). The GSK p38 inhibitors are currently being tested to prevent unwanted inflammation. A surprising observation we made was that p38 inhibition could also rejuvenate human T lymphocytes and enhance their ability to proliferate in vitro. Therefore the inhibition of p38 may block unwanted inflammatory response as well as enhance T lymphocyte reactivity. Vitamin D3 has also been shown to have anti-inflammatory effects and works in part by inhibiting p38MAPkinase activation. The use of 2 separate interventions in this project increases the likelihood of success in this study. While Losmapimod may be more specific in its anti-inflammatory action, the use of vitamin D3 is more cost effective as it is a cheap compound that is readily available.     The desirable outcome of this study is that either one or other of these compounds, that older volunteers will be treated with, will boost their response to microbial antigen challenge in the skin. This will provide proof of concept data that will lead to the exciting possibility of enhancing immunity by inhibiting the increased baseline inflammatory responses that are found during ageing. This may be a strategy to enhance immune responses in general and specifically to enhance vaccination efficacy to various diseases that is less effective in older subjects.\nHomo sapiens\nExpression profiling by high throughput sequencing\nA total of 126 samples were analysed, including normal skin (n=30), saline (6 hour; n=30), VZV (6 hour; n=32), and VZV (day2; n=34).\n',
 'Second-generation molecular subgrouping of medulloblastoma: an international meta-analysis of Group 3 and Group 4 subtypes\nWe undertook a meta-analysis based on DNA methylation patterns of Group3/4 subgroup medulloblastoma from three published studies, alongside additional unpublished tumours (total n=1501).\nHomo sapiens\nMethylation profiling by genome tiling array\nWe used 267 samples from GSE93646, 356 samples from GSE85212, 725 samples from Northcott et al., 2017 (PMID: 28726821) and 153 previously unpublished samples from the St. Jude collection. Bisulfite-converted DNA from medulloblastoma samples were hybridised to the Illumina Infinium 450k and Infinium MethylationEPIC BeadChips.\n',
 'The major risk factors for Alzheimerâ\x80\x99s disease: Age, Sex and Genes, modulate the microglia response to AÎ² plaques\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'The major risk factors for Alzheimerâ\x80\x99s disease: Age, Sex and Genes, modulate the microglia response to AÎ² plaques (KW)\nMicroglia are involved in Alzheimerâ\x80\x99s disease (AD) by adopting activated phenotypes. How ageing in the absence or presence of Î²-amyloid (AÎ²) deposition in different brain areas affects this response and whether sex and AD risk genes are involved, remains however largely unknown. Here we analyzed the gene expression profiles of more than 10,000 individual microglia cells isolated from cortex and hippocampus of male and female AppNL-G-F at 4 different stages of AÎ² deposition and in age-matched control mice. We demonstrate that microglia adopt two major activated states during normal aging and after exposure to amyloid plaques. One of the responses (activated response microglia, ARM) is enhanced in particular by amyloid plaques and is strongly enriched with AD risk genes. The ARM response is not homogeneous, as subgroups of microglia overexpressing MHC type II and tissue repair genes (Dkk2, Gpnmb, Spp1) are induced upon prolonged AÎ² exposure. Microglia in female mice advance faster in the activation trajectories. Similar activated states were also found in a second AD model and in human brain. We demonstrate that abolishing the expression of Apoe, the major genetic risk factor for AD, impairs the establishment of ARMs, while the second microglia response type, enriched for interferon response genes, remains unaffected. Our data indicate that ARMs are the converging point of multiple AD risk factors.\nMus musculus\nExpression profiling by high throughput sequencing\nRNA-Seq of cortical and hippocampal microglia in female and male C57Bl/6 wild type mice and in AppNL-G-F mice M3, M6, M12, M21 (32 experimental conditions, 2 mice per condition - data pooled).Â\n',
 'The major risk factors for Alzheimerâ\x80\x99s disease: Age, Sex and Genes, modulate the microglia response to AÎ² plaques (CDEP)\nMicroglia are involved in Alzheimerâ\x80\x99s disease (AD) by adopting activated phenotypes. How ageing in the absence or presence of Î²-amyloid (AÎ²) deposition in different brain areas affects this response and whether sex and AD risk genes are involved, remains however largely unknown. Here we analyzed the gene expression profiles of more than 10,000 individual microglia cells isolated from cortex and hippocampus of male and female AppNL-G-F at 4 different stages of AÎ² deposition and in age-matched control mice. We demonstrate that microglia adopt two major activated states during normal aging and after exposure to amyloid plaques. One of the responses (activated response microglia, ARM) is enhanced in particular by amyloid plaques and is strongly enriched with AD risk genes. The ARM response is not homogeneous, as subgroups of microglia overexpressing MHC type II and tissue repair genes (Dkk2, Gpnmb, Spp1) are induced upon prolonged AÎ² exposure. Microglia in female mice advance faster in the activation trajectories. Similar activated states were also found in a second AD model and in human brain. We demonstrate that abolishing the expression of Apoe, the major genetic risk factor for AD, impairs the establishment of ARMs, while the second microglia response type, enriched for interferon response genes, remains unaffected. Our data indicate that ARMs are the converging point of multiple AD risk factors.\nMus musculus\nExpression profiling by high throughput sequencing\nRNA-Seq of cortical and hippocampal microglia in  male APP/PS1-Apoe(null), APP/PS1, C57Bl/6 and C57Bl/6-Apoe(null) mice ;  at month 18\n',
 'Blood Biomarkers for Memory: Towards Early Detection of Risk for Alzheimer Disease, Pharmacogenomics, and Repurposed Drugs\nShort-memory dysfunction is a key early feature of Alzheimer Disease. Psychiatric patients may be at higher risk for memory dysfunction and subsequent Alzheimer due to the negative effects of stress and depression on the brain. We carried out longitudinal within-subject studies in male and female psychiatric patients to identify blood gene expression biomarkers that track short term memory as measured by the retention measure in the Hopkins Verbal Learning Test. These biomarkers were prioritized with a convergent functional genomics approach using previous evidence in the field  implicating them in Alzheimer Disease. The top candidate biomarkers were tested in an independent cohort for ability to predict state short-term memory, and trait future positive neuropsychological testing for cognitive impairment\nHomo sapiens\nExpression profiling by array\nAlzheimer disease (AD) is a clear and present danger to older adults, and has a profound socio-economic impact (refs). Existing therapies are limited in efficacy (refs). Early identification of individuals at risk may open the door to preventive approaches.  Short-term memory dysfunction is a key early feature of AD. We proposed to identify blood biomarkers that track a relevant related quantitative phenotype for short-term memory, the retention measure of recall in Hopkins Verbal Learning Test (HVLT) (Figure S1).  Previous work by our group has identified blood gene expression biomarkers that track suicidal ideation using longitudinal within-subject designs, validated them in suicide completers, and tested them in independent cohorts for ability to assess state (suicidal ideation), and ability to predict trait (future hospitalizations for suicidality) (Le-Niculescu et al. Molecular Psychiatry 2013, Niculescu et al. Molecular Psychiatry 2015) (Levey et al. Molecular Psychiatry 2016) (Niculescu et al. Molecular Psychiatry 2017). Those biomarkers were also useful in pharmacogenomic and drug repurposing analyses. We endeavored to use a similar approach to identify biomarkers for short-term memory.  We conducted our longitudinal studies in psychiatric disorder subjects, a population enriched in memory retention abnormalities, and which  may be at increased risk of AD and other aging-related disorders, due to the effects  of stress and depression (Nho et al. 2015, Rangaraju et al. 2016, al refs).  The subjects had blood gene expression data at multiple testing visits, and were phenotyped at each visit, including with HVLT. They also had electronic medical records for long term follow-up of subsequent outcomes, including future neuropsychological testing as part of standard clinical care.Diagnoses:SZ - SchizophreniaSZA - schizoaffective disorderBP - Bipolar DisorderMOOD - MoodPSYCH - Psychotic Disorder NOSMDD - Major Depressive DisorderPTSD - Posttraumatic Stress Disorder\n',
 'Single-cell multi-omic profiling of chromatin conformation and DNA methylome\nRecent advances in the development of single cell epigenomic assays have facilitated the analysis of the gene regulatory landscapes in complex biological systems. Single-cell variations of methods such as DNA methylation-sequencing and ATAC-seq hold tremendous promise for delineating distinct cell types and identifying their critical cis-regulatory sequences. Emerging evidence in recent years has shown that in addition to cis-regulatory sequences, dynamic regulation of 3D chromatin conformation is a critical mechanism for the modulation of gene expressions during development and disease. While assays for the investigation of single-cell 3D chromatin structure have been developed, cell-type specific chromatin conformation in primary human tissues has not been extensively explored. It remains unclear whether single-cell Chromatin Conformation Capture (3C) or Hi-C profiles are suitable for cell type identification and allow the reconstruction of cell-type specific chromatin conformation maps. To address these challenges, we have developed a multi-omic method single-nucleus methyl-3C sequencing (sn-m3C) to profile chromatin conformation and DNA methylation from the same cell. We have shown that bulk m3C and sn-m3C accurately capture chromatin organization information and robustly separate mouse cell types. We have developed a fluorescent-activated nuclei sorting strategy based on DNA content that eliminates nuclei multiplets caused by crosslinking. The sn-m3C-seq method allows high-resolution cell-type classification using two orthogonal types of epigenomic information and the reconstruction of cell-type specific chromatin conformation maps.\nMethylation profiling by high throughput sequencingOther\nchromatin conformation and DNA methylome\n',
 'Large-scale serum microRNA profiling for developing an esophageal squamous cell carcinoma detection model\nA serum miRNA combination could be a powerful classifier for the detection of esophageal squamous cell carcinoma.\nHomo sapiens\nNon-coding RNA profiling by array\nSerum microRNA profiles of 5531 samples, which consist of 566 of esophageal squamous cell carcinoma and 4965 of non-cancer controls.\n',
 'Functional heterogeneity within the developing zebrafish epicardium\nThe epicardium is essential during cardiac development, homeostasis and repair and yet fundamental insights into its underlying cell biology, notably epicardium formation, lineage heterogeneity and functional cross-talk with other cell types in the heart, is currently lacking. In this study, we investigated epicardial heterogeneity and the functional diversity of discrete epicardial subpopulations in the developing zebrafish heart. Single-cell RNA-sequencing uncovered three epicardial subpopulations with specific genetic programmes and distinctive spatial distribution within the developing heart. Perturbation of unique gene signatures uncovered distinct functions associated with each subpopulation and established novel epicardial roles in cell adhesion, migration, and chemotaxis as a mechanism for recruitment of leukocytes into the heart. This work elucidates the mutual spatiotemporal relationships between different epicardial subpopulations and assigns unique function to each during cardiac development. Understanding which mechanisms cells employ to establish a functional epicardium and to communicate with other cardiovascular cell types during development will bring us closer to repairing cellular relationships that are disrupted during cardiovascular disease.\nDanio rerio\nExpression profiling by high throughput sequencing\nSingle cell RNA sequencing of FACS purified larval cardiac zebrafish cells\n',
 'Risk prediction models for dementia constructed by supervised principal component analysis using miRNA expression data\nAlzheimerâ\x80\x99s disease (AD) is the most common subtype of dementia, followed by Vascular Dementia (VaD), and Dementia with Lewy Bodies (DLB). Recently, microRNAs (miRNAs) have received a lot of attention as the novel biomarkers for dementia. Here, using serum miRNA expression of 1,601 Japanese individuals, we investigated potential miRNA bio- markers and constructed risk prediction models, based on a supervised principal component analysis (PCA) logistic regression method, according to the subtype of dementia. The final risk prediction model achieved a high accuracy of 0.873 on a validation cohort in AD, when using 78 miRNAs: Accuracy = 0.836 with 86 miRNAs in VaD; Accuracy = 0.825 with 110 miRNAs in DLB. To our knowledge, this is the first report applying miRNA-based risk pre- diction models to a dementia prospective cohort. Our study demonstrates our models to be effective in prospective disease risk prediction; and with further improvement may contribute to practical clinical use in dementia.\nHomo sapiens\nNon-coding RNA profiling by array\n1,601 serum samples (1,021 AD cases, 91 VaD cases, 169 DLB cases, 32 Mild Cognitive Impairment: MCI, and 288 NC).\n',
 'Discovery and characterization of variance QTLs in human induced pluripotent stem cells\nQuantification of gene expression levels at the single cell level has revealed that gene expression can vary substantially even across a population of homogeneous cells. However, it is currently unclear what genomic features control variation in gene expression levels, and whether common genetic variants may impact gene expression variation. Here, we take a genome-wide approach to identify expression variance quantitative trait loci (vQTLs). To this end, we generated single cell RNA-seq (scRNA-seq) data from induced pluripotent stem cells (iPSCs) derived from 53 Yoruba individuals. We collected data for a median of 95 cells per individual and a total of 5,447 single cells, and identified 241 mean expression QTLs (eQTLs) at 10% FDR, of which 82% replicate in bulk RNA-seq data from the same individuals. We further identified 14 vQTLs at 10% FDR, but demonstrate that these can also be explained as effects on mean expression. Our study suggests that dispersion QTLs (dQTLs), which could alter the variance of expression independently of the mean, have systematically smaller effect sizes than eQTLs. We estimate that at least 300 cells per individual and 400 individuals would be required to have modest power to detect the strongest dQTLs in iPSCs. These results will guide the design of future studies on understanding the genetic control of gene expression variance.\nHomo sapiens\nExpression profiling by high throughput sequencingOther\nThe goal of our study was to identify quantitative trait loci associated with gene expression variance across cells (vQTLs). Using the Fluidigm C1 platform, we isolated and collected scRNA-seq from 7,585 single cells from induced pluripotent stem cell (iPSC) lines of 54 Yoruba in Ibadan, Nigeria (YRI) individuals. We used unique molecular identifiers (UMIs) to tag RNA molecules and account for amplification bias in the single cell data (Islam et al., 2014). To estimate technical confounding effects without requiring separate technical replicates, we used a mixed-individual plate study design. The key idea of this approach is that having observations from the same individual under different confounding effects and observations from different individuals under the same confounding effect allows us to distinguish the two sources of variation (Tung et al., 2017).\n',
 'Serum miRNA-based risk prediction for stroke\nThe serum miRNAs were correlated with predicted the incidence of stroke.\nHomo sapiens\nNon-coding RNA profiling by array\nSerum microRNA profiles of 1785 samples, which consist of 173 of CVD patients, 1612 of non-CVD control.\n',
 'In vivo molecular signatures of severe dengue infection revealed by viscRNA-Seq\nDengue virus infection can result in severe symptoms including shock and hemorrhage, but an understanding of the molecular correlates of disease severity is lacking. Bulk transcriptomics on blood samples are difficult to interpret because the blood is composed of different cell types that may react differently to virus infection. Dengue virus RNA can be detected in human plasma, however identifying the cells carrying dengue virus through the bloodstream in vivo has proven challenging. Here we used our recently developed viscRNA-Seq approach to profile transcriptomes of thousands of single blood peripheral mononuclear cells from 6 human subjects with dengue fever and severe dengue, as well as to characterize the cell types associated with dengue virus in the human blood. We found that although no bulk transcriptome marker for severe dengue exists, the expression of MX2 in naive B cells, of CD163 in CD14+/CD16+ monocytes and of other genes in specific cell types is highly predictive for severe dengue. We detected virus-associated cells in the blood of two severe dengue patients with high viral load and discovered the majority of these to be B cells expressing germline IgM or IgD immunoglobulin chains and naive markers but also showing signs of activation and expression of CD69, CXCR4, and other surface receptors. In bystander B cells we detected signs of strong immune activation, parallel hypersomatic evolution and, in one severe degue subject, an anomalously large clone of highly mutated, IgG1 plasmablasts that could be reactive to dengue virus. This study presents a high-resolution molecular exploration into dengue virus infection in humans and can be generalized to any RNA virus.\nHomo sapiens\nExpression profiling by high throughput sequencing\nBlood cells from dengue virus infected human patients were subjected to virus-inclusive single cell RNA-Seq.\n',
 "miRNA and colorectal cancer:  associations with tumor phenotype and survival\nMicroRNAs (miRNA) are a class of small regulatory RNAs that mediate post-transcriptional silencing of specific target mRNAs.  Data suggest the importance of miRNAs to cancer development and possibly to survival.  Our overall hypothesis is that miRNA expression is unique to tumor molecular phenotype; that miRNA expression levels at time of diagnosis predicts survival; and that miRNA expression is associated with inflammation-related genetic and lifestyle factors key to colorectal cancer (CRC). This study takes a two pronged approach to addressing our hypotheses.  While we propose to validate previously identified miRNAs that have been identified as associated with CRC (either by differential expression or from assessment of mutations), we will add to the field through discovery of new and important associations that may be unique to specific molecular phenotypes, to polyp to cancer progression, and  to survival.  We have analyzed the expression of 2006 human miRNAs using data derived from tumor and paired normal tissue at time of diagnosis from:  1975 people with incident colon cancer or rectal cancer and 290 polyps from colon and rectal cases (included in this study) who reported a prior polyp.  MiRNA was obtained from dissected paraffin-embedded tissue and assessed using an Agilent microarray platform.  We intend to extend our validation of previously identified mutated miRNAs and differentially expressed miRNAs to determine if these alterations are associated with specific tumor molecular phenotype (TP53, KRAS2, CIMP+, and for colon tumors MSI+), inflammation-related factors, clinical factors and survival.  We will identify associations with miRNAs that are related to specific molecular phenotypes, with drivers in the carcinogenic process, and with clinical features and survival.  These miRNAs will be validated using targeted Agilent Platform. Associations will be tested based on differential expression for both individual and groups of miRNAs using recent extensions of several statistical methods including ANOVA, logistic regression, and Cox proportional hazards models.  Our sample size allows for both a training and validation component, and provides sufficient statistical power to meet the study goals.  MiRNAs that are differentially expressed in polyps and in subsequent tumors will provide new insights into targets for screening and treatment as well as provide support that miRNAs function as the â\x80\x9cdriverâ\x80\x9d in the carcinogenic process.  Testing of mutated miRNAs identified from sequencing in conjunction with tumor phenotype, clinical, and survival data will further validate the importance of these miRNAs, and provide insight as to which CRC molecular pathway the miRNAs function.  Our rich dataset of lifestyle, genetic, clinical and prognosis, and tumor molecular phenotype on 1975 CRC and paired normal tissue allows us to examine factors that are associated with miRNA expression and mutation in a large sample of population-based cases in the most cost-efficient manner possible. The miRNAs identified in these analyses will elucidate pathways important in the etiology of CRC and will provide insight into potential targets for screening and treatment.MiRNAs are small, non-protein-coding RNA molecules that regulate gene expression either by post-transcriptionally suppressing mRNA translation or by mRNA degradation.  We examine differentially expressed miRNAs in colorectal carcinomas, adenomas, and normal colonic mucosa. Data come from population-based studies of colorectal cancer conducted in Utah and the Kaiser Permanente Medical Care Program.  A total of 1893 carcinoma/normal paired samples and 290 adenoma tissue samples were run on the Agilent Human miRNA Microarray V19.0 which contained 2006 miRNAs.  We tested for significant differences in miRNA expression between paired carcinoma/adenoma/normal colonic tissue samples.  Fewer than 600 miRNAs were expressed in >80% of people for colonic tissue; of these 86.5% were statistically differentially expressed between carcinoma and normal colonic mucosa using a False Discovery Rate of 0.05.  Roughly half of these differentially expressed miRNAs showed a progression in levels of expression from normal to adenoma to carcinoma tissue.  Other miRNAs appeared to be altered at the normal to adenoma stage, while others were only altered at the adenoma to carcinoma stage or only at the normal to carcinoma stage.    Evaluation of the Agilent platform showed a high degree of repeatability (r=0.98) and reasonable agreement with the NanoString platform.  Our data suggest that miRNAs are highly dysregulated in colorectal tissue among individuals with colorectal cancer; the pattern of disruption varies by miRNA as tissue progresses from normal to adenoma to carcinoma.\nHomo sapiens\nNon-coding RNA profiling by array\nPopulation-based ascertainment of all diagnosed cases of colorectal cancer from Utah and Northern California Kaiser.Please note that the 'characteristics: AJCC_stage' values for each sample have been updated on July 5th, 2018.\n",
 'High-sensitive circulating microRNA panel for early detection of hepatocellular carcinoma\nA serum miRNA combination could be a powerful classifier for the detection of hepatocellular carcinoma.Keywords: Non-coding RNA profiling by array\nHomo sapiens\nNon-coding RNA profiling by array\nSerum microRNA profiles of 1817 samples, which consist of 345 with hepatocellular carcinoma,  46 patients with chronic hepatitis, 93 patients with liver cirrhosis, and 1033 non-cancer individuals.\n',
 'Smart-seq2 analysis of larval zebrafish habenula from the gng8-GFP transgenic line\nThe identification of cell types and marker genes is critical for dissecting neural development and function, but the size and complexity of the brain has hindered the comprehensive discovery of cell types. We combined single-cell RNA-seq with anatomical brain registration to create a comprehensive map of the zebrafish habenula, a conserved forebrain hub involved in pain processing and learning. Single-cell transcriptomes of ~13000 habenular cells (>4x coverage) identified 18 neuronal types and dozens of marker genes. Registration of marker genes onto a common reference atlas created a rich resource for anatomical and functional studies and enabled the mapping of active neurons onto neuronal types following aversive stimuli. Strikingly, despite brain growth and functional maturation, cell types were retained between the larval and adult habenula. This study provides a gene expression atlas to dissect habenular development and function and offers a general framework for the comprehensive characterization of other brain regions.\nDanio rerio\nExpression profiling by high throughput sequencing\ngng8-GFP zebrafish heads were dissected, dissociated and FAC sorted into 96 well plates. Single cell libraries were generated in batches of 384 cells using Smart-seq2. A total of 22 gng8-GFP fish were dissected in 3 batches and 384 cells were processed from each using Smart-seq2.\n',
 '10X genomics analysis of adult and larval habenula from the gng8-GFP transgenic line\nThe identification of cell types and marker genes is critical for dissecting neural development and function, but the size and complexity of the brain has hindered the comprehensive discovery of cell types. We combined single-cell RNA-seq with anatomical brain registration to create a comprehensive map of the zebrafish habenula, a conserved forebrain hub involved in pain processing and learning. Single-cell transcriptomes of ~13000 habenular cells (>4x coverage) identified 18 neuronal types and dozens of marker genes. Registration of marker genes onto a common reference atlas created a rich resource for anatomical and functional studies and enabled the mapping of active neurons onto neuronal types following aversive stimuli. Strikingly, despite brain growth and functional maturation, cell types were retained between the larval and adult habenula. This study provides a gene expression atlas to dissect habenular development and function and offers a general framework for the comprehensive characterization of other brain regions.\nDanio rerio\nExpression profiling by high throughput sequencing\nFor larval dataset, multiple gng8-GFP labeled zebrafish brains were dissociated and habenular cells FAC sorted into PBS+1%BSA and immediately loaded into 10X chromium for cell capture. For adult dataset, 5-6 habenulae were cleanly dissected out, dissociated, filtered in PBS+BSA and immediately captured in droplets with 10x platform. Both libraries were then constructed using standard instrutions of the 10x plaform.\n',
 'Integrated extracellular microRNA profiling for ovarian cancer screening\nA serum miRNA combination could be a powerful classifier for the detEsophageal Cancertion of patients with ovarian cancer.\nHomo sapiens\nNon-coding RNA profiling by array\nSerum microRNA profiles of 4046 woman samples, which consist of 333 of ovarian cancers, 66 of borderline ovarian tumors, 29 of benign ovarian diseases, 2759 of non-cancer controls, and 859 of the other solid cancers.\n',
 'Comprehensive identification and spatial mapping of habenular neuronal types using single cell RNAseq\nThis SuperSeries is composed of the SubSeries listed below.\nDanio rerio\nExpression profiling by high throughput sequencing\nRefer to individual Series\n',
 'The profile of exosomal and cellular miRNA in ovarian cancer cell lines\nTo check the profile of exosomal and cellular miRNA in ovarian cancer cell lines, total RNA were extracted from exosomes and cells. Thirteen ovarian cancer cell lines (A2780, ES-2, CAOV3, SKOV3, OV-90, OAW42, MCAS, COV362, RMG-1, RMUG-S, KURAMOCHI, NIH-OVCAR3 and A2780cis) were investigated, and HOSE1, HOSE2 and HOSE3 (human ovarian surface epithelim cell lines) were used as control.\nHomo sapiens\nNon-coding RNA profiling by array\nTo check the profile of exosomal and cellular miRNA in ovarian cancer cell lines, total RNA were extracted from exosomes and cells. Microarray analysis was performed using 50 ng of exosomal miRNAs and 100 ng of cellular miRNAs.\n',
 'RNA-seq of Arabidopsis thaliana seedlings growing in the presence of a bacterial synthetic community under full and low phosphate conditions\nThis study evaluates the transcriptome of Arabidopsis thaliana seedlings growing in the presence of a diversity of synthetic bacteria communities under different phosphate availability.\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nArabidopsis seeds (wild-type Col-0) were germinated on Johnson medium containing 0.5 % sucrose with either 1 mM Pi (in the form of KH2PO4), or no Pi added. Plants were then transferred to 30 ÂµM Pi (low) or 100 ÂµM Pi in the absence of sucrose alone or with one of the synthetic communities.Please note that the different values (e.g. I1I2, I2I3 etc.) in the \'sample characterics: bacteria\' field represents different "synthetic communities" (ie groups of bacteria applied together). There are 9 blocks (P1,P2,P3,I1,I2,I3,N1,N2,N3) each of 8-9 bacterial strains, and each synthetic community is made by combining two blocks; for example, samples that say P1P2 were grown in the presence of a synthetic community that had all the strains from both P1 and P2.\n',
 'The biological age linked with oxidative estress modifies breast cancer aggressiveness\nThe incidence of breast cancer increases with age until menopause, and breast cancer is more aggressive in younger women. The existence of epidemiological links between breast cancer and aging indicates that both processes possibly share some common mechanisms of development. Oxidative stress is associated with both cancer susceptibility and aging. Here we observed that ERBB2-positive breast cancer, developed from genetically different ERBB2-positive transgenic mice generated by a backcross, is more aggressive in younger than in older mice, similar to what occurs in humans. In this cohort, we estimated the oxidative biological age using a mathematical model that integrated several subphenotypes directly or indirectly related to oxidative stress. This model included the serum levels of HDL-cholesterol and magnesium that were determined at a disease-free stage, and total AKT1 and glutathione concentrations in the liver at the time of necropsy. Later we defined the grade of aging due to oxidative stress as the increment of aging, which was the difference between the predicted biological age and the chronological age. This comparison permitted the identification of the biologically younger mice with less oxidative stress and older mice with more oxidative stress than would be expected according to their chronological age. Interestingly, biologically older mice developed more aggressive breast cancer than the biologically younger mice. We identified genomic regions on chromosomes 2 and 15 that were linked to the grade of oxidative aging. The levels of expression of Zbp1 located on chromosome 2, a gene related to necroptosis and inflammation, positively correlated with the grade of oxidative aging and tumour aggressiveness. This study shows part of the complex interactions between breast cancer and aging.\nExpression profiling by array\nWe have studied differential gene expression from livers in mice with different susceptibility with breast cancer. We evaluated 50 livers from mice with extreme values of grade of aging determined by a multivariate model.\n',
 'Single-cell Multi-omics Sequencing and Analyses of Human Colorectal Cancer\nAlthough genomic instability, epigenetic abnormality, and gene expression dysregulation are hallmarks of colorectal cancer, these features have not been simultaneously analyzed at single-cell resolution. Using optimized single-cell multi-omics sequencing together with multi-regional sampling of the primary tumor, lymphatic and distant metastases, we provide insights beyond intratumoral heterogeneity. Genome-wide DNA methylation levels were relatively consistent within a single genetic sub-lineage. The genome-wide DNA demethylation patterns of cancer cells were consistent in all 10 sequenced patients. Our work demonstrates the feasibility of reconstructing genetic lineages, and tracing their epigenomic and transcriptomic dynamics with single-cell multi-omics sequencing.\nHomo sapiens\nExpression profiling by high throughput sequencingMethylation profiling by high throughput sequencing\nSingle cell RNA-seq and Bisulfite-seq on whole cells or by TrioSeq2.[scTrioSeq2Rna and scTrioSeq2Met Samples]Sample Title structure: Library_PatientID_SamplingPositions_CellIDSamplingPositions abbreviations:PT:  Primary TumorLN:  Lymph Node metastasisML:  Liver MetastasisMP:  Post-treatment Liver Metastasis\n',
 'Single Cell Methylome Signatures Reveal Distinct Neuronal Populations and Regulatory Elements in the Mammalian Cortex\nThe mammalian brain contains diverse neuron cell types with distinct physiological, morphological and molecular characteristics. However, a comprehensive assessment of the epigenetically distinct neuronal classes is currently missing. Cytosine DNA methylation is a stable epigenetic mark that distinguishes neuron types and marks gene regulatory elements. We developed an efficient single-nucleus methylome sequencing approach that allows robust high-throughput neuron-type classification. We generated >6,000 single-nucleus methylomes and identified 16 mouse and 21 human neuronal subpopulations in the frontal cortex. Both CG and non-CG methylation exhibited cell type-specific distributions that recapitulate and extend findings from single neuron transcriptome profiling. Moreover, we found approximately 500,000 neuron-type-specific regulatory elements showing strong differential methylation in mouse and human cortex. Distinct methylation signatures identified an unique human Parvalbumin-expressing inhibitory sub-type and a layer 6 specific excitatory sub-type in mouse. Comparative epigenomic analysis showed stronger conservation of gene regulatory elements in inhibitory compared with excitatory neurons. These findings demonstrate the utility of single nucleus methylome profiling for both expanding the atlas of brain cell types and identifying regulatory elements that potentially drive these differences.\nMethylation profiling by high throughput sequencingOther\nSingle Cell Methylome analysis of mouse and human frontal cortex\n',
 'Probing Plasmodium falciparum sexual differentiation at the single cell level\nMalaria parasites go through major transitions during their complex life cycle, yet the underlying differentiation pathways remain obscure. Here we apply single cell transcriptomics to unravel events that initiate sexual development in preparation for transmission of the parasite from human to mosquito. This proof-of-concept study provides a template to capture transcriptional diversity in heterogeneous parasite populations, with major implications for our understanding of parasite biology and the ongoing malaria elimination campaign.\nPlasmodium falciparum NF54\nExpression profiling by high throughput sequencing\nPlasmodium-infected human erythrocytes were single-cell sorted into 384-well plates at three separate time points after LysoPC depletion gametocyte induction (4, 8, 12 hours post induction). Two rows (total of 48 wells) of each 384-well plate were reserved for control samples (no induction). A total of 1152 wells were sequenced, with 144 control wells, and 1008 induced wells.\n',
 'Time-resolved transcriptome analysis with genetic perturbations reveals a critical time window for effective plant immunity\nPathogen invasion in plants is associated with transcriptional reprogramming. Enigmatically, plants induce similar transcriptome responses upon infection by virulent or avirulent pathogens. This renders the importance of transcriptional reprogramming for immunity obscure. Here, using RNA-seq, we generate time-series transcriptome data coupled with genetic perturbations to reveal temporal dynamics upon infection by virulent or avirulent strains of a bacterial pathogen, Pseudomonas syringae, in Arabidopsis thaliana. Fast and sustained transcriptional reprogramming occurs upon infection with avirulent strains while virulent strain infection leads to a slower response with comparable gene expression patterns and magnitudes. Importantly, transcriptome analysis of resistant and susceptible mutants responding to avirulent strains links delayed transcriptional reprogramming to compromised immunity. Taken together, our results pinpoint the early critical time window of transcriptional reprogramming for establishing effective immunity against the bacterial pathogen.\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nLeaves of Col-0 and all the single, double, triple and quadruple mutants of dde2-2, ein2-1, pad4-1, sid2-2 were syringe-infiltrated with mock (water) or suspensions of Pseudomonas syringae pv. tomato DC3000 (Pto DC3000) carrying an empty vector (pLAFR), Pto DC3000 carrying AvrRpt2, or Pto DC3000 carrying AvrRpm1 at the OD600 of 0.001. Similarly, leaves of the rpm1-3 rps2-101C mutant plants were inoculated with mock, Pto DC3000 carrying AvrRpt2 or Pto DC3000 carrying AvrRpm1. Three fully-expanded leaves (leaves 7-9) from three different plants were harvested as a single biological replicate at 1, 2, 3, 4, 6, 9, 12, 16, 20, 24, 36, 48 hours post inoculation (hpi). To generate three biological replicates, three independent experimental trials were carried out, in which plant positions within pots and growth chambers were randomized in order to avoid undesirable systematic effects. For the statistcal analysis, 348 samples (M001-M348) were used.\n',
 'RNA-seq of Solanum lycopersicum cv. M82 and S. pennellii introgression line 3-5 day old root tip tissue\nThe goal of this study was to perform RNA-seq expression analysis on Solanum lycopersicum cv. M82 X S. pennellii introgression lines, deriving expression Quantitative Trait Loci which were analyzed together with pre-existing genomic and phenotypic data to define genes and regulatory pathways controlling tomato root development and observed natural variation.  We completed the RNAseq expression profiling analysis and developed a tool to display this information graphically in collaboration with Nicholas Provart at the University of Toronto: http://bar.utoronto.ca/efp_tomato/cgi-bin/efpWeb.cgi?dataSource=ILs_Root_Tip_Brady_Lab To identify candidate genes and pathways we focussed on one root growth trait, root growth angle, and identified two statistically significant genomic regions within tomato root growth angle QTL containing two candidate genes that likely control the gravitropic setpoint angle (CDC73 and PAP27), both of which are conserved between Arabidopsis and tomato, and which we tested using transgenic lines of the Arabidopsis orthologs.  A possible regulatory role for suberin in root growth angle control was also identified.\nExpression profiling by high throughput sequencing\nSeedlings of S. lycopersicum (cv. M82), S. pennellii (LA0716), and 75 introgression lines between those parents were grown according to (Ron et al., 2013). Plates were placed vertically in a rack in a randomized block design and the experiments were carried out in a growth chamber with a 16:8 light:dark cycle at 22ÂºC and 50-75% humidity with a light intensity of 100Î¼E. The germination date was monitored for each seed. Immediately prior to harvesting root tissue for RNA-seq, each replicate of each genotype had between 2 and 5 plates, all of which were scanned with a Epson Perfection V700 photo flatbed scanner into 24-bit RGB TIF image files at 300 dpi. Images were analyzed with the imageJ program (Rasband 1997-2014) and custom macros. Based on germination date and scan date, seedling age in days after germination was determined, and primary roots of seedlings that were 3 or 4 days post-germination were traced with the imageJ polyline tool, following the root centerline from the top of the root (at start of hairs and change from green to white) to the tip. The coordinates of the polylines were recorded by the macros in data tables saved as files, along with metadata including genotype, plate ID, plating date, germination day of each seedling, plate harvest date, measurer, and two flags for each root: â\x80\x9ccollideâ\x80\x9d was true if the root contacted or crossed another root, and â\x80\x9calongâ\x80\x9d was true if the root contacted another root and grew along it for any distance. The polylines in the vetted data were analyzed to measure root length and angle using multiple traits (Figure 1A). Additional data was calculated from the metadata: root age (days from germination to harvest), â\x80\x9cgermination ageâ\x80\x9d (days from plating to germination), â\x80\x9cplating ageâ\x80\x9d (days from plating to harvest), and biological replicate number (based on plating date; replicates were named REP2 through REP5, and later replicates for fixing genotype errors were named REPN2 through REPN5 and REPX1 through REPX6). Plates were opened only once, at harvest time, to ensure the same growth conditions for all seedlings. Sampling occurred mid-afternoon. Ten segments (1cm) of the root tip from each genotype replicate were cut and immediately placed into a labeled 2ml tube containing silica beads and immersed in liquid nitrogen. Samples were stored at â\x88\x9280ÂºC until library preparation. Four replicates of each of the 77 genotypes were grown. Between 7 and 14 root tips, 3 to 4 days post-germination, were collected and libraries were created using a modified Kumar et. al. protocol. Library enrichment was done using 12 cycles. Each library was barcoded, using a unique bar code for each IL, but the same barcode for all replicates of an IL.  The libraries of each replicate were pooled, and the four pools were purified with Ampure beads. The library pooling was tested using a Bioanalyzer. Each pool was sequenced in 50 bp single-end format on a total of 5 lanes of a HiSeq 2000 platform (Illumina Inc. San Diego, CA, USA).  Two replicates were lost during library creation, S. pennellii #3 and IL7-5 #3.\n',
 "Molecular Signatures Underlying Selective Regional Vulnerability to Alzheimer's Disease\nAlzheimer's disease (AD) is the most common form of dementia, characterized by progressive cognitive impairment and neurodegeneration as a result of abnormal neuronal loss. To elucidate the molecular systems associated with AD, we characterized the gene expression changes associated with multiple clinical and neuropathological traits in 1,053 postmortem brain samples across 19 brain regions from 125 persons dying with varying severities of dementia and variable AD-neuropathology severities.\nHomo sapiens\nExpression profiling by array\n125 human brains were accessed from the Mount Sinai/JJ Peters VA Medical Center Brain Bank (MSBB). This brain resource was assembled after applying stringent inclusion/exclusion criteria and represents the full spectrum of clinical and neuropathological disease severity in the absence of discernable non-AD neuropathology. RNA samples from 19 brain regions isolated from the 125 MSBB specimens were collected and profiled using Affymetrix Genechip microarrays. There were 50 to 60 subjects per brain region with varying degrees of AD pathological abnormalities.\n",
 "Epigenomic and genome structural diversity in a worldwide collection of Arabidopsis thaliana\nEpigenetic variation can impact gene transcription and may play roles in phenotypic diversity and adaptation. Here we report 1,107 high quality single-base resolution methylomes, and 1,210 transcriptomes from the 1001 Arabidopsis Genomes population. Analyses reveal strong effects of geographic origin on average DNA methylation levels, alterations of gene expression by epialleles and a highly complex genetic basis for DNA methylation. Physical genome maps for nine of the most diverse accessions revealed how transposable elements and other structural variations shaped the epigenome to allow rapid adaptation to environmental changes, with strong emphasis on disease resistance. Analysis of the cistromes and epicistromes in these accessions revealed a significant association between both methylation and nucleotide variation and the conservation of transcription factor binding sites. The Arabidopsis thaliana 1001 Epigenomes Project now provides a comprehensive resource to help further understand how epigenetic variation contributes to both molecular and phenotypes in natural populations of the most widely studied reference plant.\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nRNA-seq profiling of 728 Arabidopsis thaliana accessionsPlease note that the sample 'Cod-0 (9836)' is missing from the data columns in the normalized expression matrix  ('ath1001_tx_norm_2016-04-21-UQ_gNorm_normCounts_k4.tsv') since the read count distribution from this sample was incompatible with the upper-quartile normalization method applied to all the samples, therefore excluded  from the expression matrix.\n",
 'Single Cell RNA-seq Study of Midbrain and Dopaminergic Neuron Development in Mouse, Human, and Stem Cells\nIn order to get a better molecular understanding of human midbrain development, this study defines cell types of the ventral midbrain in both human and mouse as well as stem cell-derived dopaminergic neuron preparations, and reveals the temporal dynamics of key lineages across development and the adult.\nExpression profiling by high throughput sequencing\nSingle cells according to Samples list consisting of: Human embryo ventral midbrain cells between 6 and 11 weeks of gestation, mouse ventral midbrain cells at six developmental stages between E11.5 to E18.5, Th+ neurons at P19-P27, and putative dopaminergic neurons at P28-P56 FACS-sorted from Slc6a3-Cre/tdTomato mice. Stem cells (iPSCs and ESCs) preparation at different stages of differentiation towards Th+ neurons.\n',
 'A novel combination of serum microRNAs for detecting breast cancer in the early stage\nMicroRNAs (miRNAs), which are stably present in serum, have been reported to be potentially useful for detecting cancer. In the present study, we examined the expression profiles of serum miRNAs in large cohorts to identify the miRNAs that can be used to detect breast cancer in the early stage. We comprehensively evaluated serum miRNA expression profiles using highly sensitive microarray analysis. A total of 1,280 serum samples of breast cancer patients stored in the National Cancer Center Biobank were used. Additionally, 2,836 serum samples were obtained from non-cancer controls and 514 from patients with other types of cancers or benign diseases. The samples were divided to a training cohort including non-cancer controls, other cancers and breast cancer and a test cohort including non-cancer controls and breast cancer. The training cohort was used to identify a combination of miRNAs that detect breast cancer, and the test cohort was used to validate that combination. miRNA expression was compared between breast cancer and non-breast cancer serum , and a combination of five miRNAs (miR-1246, miR-1307-3p, miR-4634, miR-6861-5p, and miR-6875-5p) was found to detect breast cancer. This combination had a sensitivity of 97.3%, specificity of 82.9%, and accuracy of 89.7% for breast cancer in the test cohort Additionally, the combination could detect breast cancer in the early stage (sensitivity of 98.0% for T0).\nHomo sapiens\nNon-coding RNA profiling by array\n1280 breast cancer serums (74 in training cohort, 1206 in test cohort), 54 benign breast diseases serums in test cohort, 2836 non-cancer control serums (1493 in training cohort, 1343 in test cohort), 514 non-breast benign diseases serums in training cohort. 150 of the non-cancer control serums in training cohort and 412 of the non-breast benign diseases serums in training cohort have been uploaded previously and are avaialable under GSE59856 (http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE59856).\n',
 'Site-specific methylation and acetylation of lysine residues in the C-terminal domain (CTD) of RNA polymerase II\nDynamic modification of the carboxy-terminal domain (CTD) of RNA polymerase II (RNAPII) regulates transcription-coupled processes in eukaryotes. The CTD in mammals is composed of 52 heptad-repeats with the consensus sequence Y1-S2-P3-T4-S5-P6-S7. Repeats in the distal part of CTD deviate from the consensus sequence and have frequently replaced serine at position 7 by lysine residues (K7). Mass spectrometry analysis revealed modification of K7 residues in heptad-repeats 39, 42, and 47/49 by acetylation and in heptad-repeats 38, 39, 40, 42, and 47/49 by mono-, di-, or trimethylation. Notably, acetylated as well as di- and tri-methylated K7 residues were found exclusively in phosphorylated CTD peptides, while mono-methylated K7 residues occurred also in non-phosphorylated CTD peptides. Methylation of K7 residues was further studied with the monoclonal antibody (mAb) 1F5, which recognizes mono- and di-methylated K7 in CTD. ChIP experiments revealed high levels of K7 methylation at the transcriptional start site of genes. The low levels of K7 methylation in the body of genes results from impairment of epitope recognition in hyper-phosphorylated CTD by mAb 1F5. In agreement with this notion, phosphatase treatment of hyper-phosphorylated CTD or treatment of cells with kinase inhibitor flavopiridol restored the reactivity of mAb 1F5 towards methylated K7 residues in CTD. We conclude that methylation and acetylation of K7 residues further expand the mammalian CTD code and potentially contribute to regulation of gene expression.\nHomo sapiens\nGenome binding/occupancy profiling by high throughput sequencing\nThis study was performed in a human Raji cell line. It contains ChIP-seq data for CTD Lysine mono- and di-methylated (one replicate). Other samples used in this study can be found at GSE52914.\n',
 'Effect of Chromomethylase-2 and Dicer-like-3 knockouts on DNA methylation\nWhole genome bisulfite sequencing was performed on Chromomethylase-2 and Dicer-like-3 knockout mutants in order to confirm the results from genome wide association mapping and to identify the respective genomic regions that they target.\nArabidopsis thaliana\nMethylation profiling by high throughput sequencing\nBisulfite sequencing of knockout mutants and WT controls\n',
 'Population epigenomics of Swedish Arabidopsis\nPlants display remarkable developmental and phenotypic plasticity in order to adapt to their environment. It has long been postulated that epigenetics plays a key role in these processes, but with one or two exceptions, solid evidence for the role of epigenetic variation in these processes is lacking. A key impediment to understanding these processes is the lack of information on the extent of epigenetic variation and how it relates to genetic and phenotypic variation in natural population, both over the lifecycle of an individual, and over evolutionary time. Here we show that genetic variants under selection in the north of Sweden appear to drive variation in DNA methylation, which in turn is highly correlated with local climate. Selective sweeps and genetic variants associated with adaptation to the local environment have previously been identified within the Swedish Arabidopsis population. Our finding that they harbour variants responsible for climate associated epigenetic variation strongly supports the role of epigenetic processes in local adaptation. These findings provide a basis for further dissecting the role of epigenetics in local adaptation at the molecular level\nArabidopsis thaliana\nMethylation profiling by high throughput sequencing\nBisulfite sequencing of 113 F2 crosses between T550 and Brosarp-11-135.\n',
 'Reinstating transcriptome plasticity and memory function in mouse models for cognitive decline\nThis SuperSeries is composed of the SubSeries listed below.\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencingExpression profiling by high throughput sequencingNon-coding RNA profiling by high throughput sequencing\nRefer to individual Series\n',
 "small RNA-Seq of the CA1 hippocampal subregion from 3 month and 20 month old animals treated orally with vehicle or SAHA\nAging and increased amyloid burden are major risk factors for cognitive diseases such as Alzheimer's Disease (AD). An effective therapy does not yet exist. Here we use mouse models for age-associated memory impairment and amyloid deposition to study transcriptome and cell type-specific epigenome plasticity at the systems level in the brain and in peripheral organs. We show that at the level of epigenetic gene-expression aging and amyloid pathology are associated with inflammation and impaired synaptic function in the hippocampal CA1 region. While inflammation is associated with increased gene-expression that is linked to a subset of transcription factors, de-regulation of plasticity genes is mediated via different mechanisms in the amyloid and the aging model. Amyloid pathology impairs histone-acetylation and decreases expression of plasticity genes while aging affects differential splicing that is linked to altered H4K12 acetylation at the intron-exon junction in neurons but not in non-neuronal cells. We furthermore show that oral administration of the clinically approved histone deacetylase inhibitor Vorinostat not only restores spatial memory, but also exhibits an anti-inflammatory action and reinstates epigenetic balance and transcriptional homeostasis at the level of gene expression and exon usage.  This is the first systems-level investigation of transcriptome plasticity in the hippocampal CA1 region in aging and AD models and of the effects of an orally dosed histone deacetylase inhibitor. Our data has important implications for the development of minimally invasive and cost-effective therapeutic strategies against age-associated cognitive decline. In fact, our data strongly suggest to test Vorinostat in patients suffering from AD.\nMus musculus\nNon-coding RNA profiling by high throughput sequencing\nsmall RNA profile from aged animals (hippocampal CA1 region) treated with oral vehicle or SAHA for 4 weeks\n",
 "mRNA-Seq of the CA1 hippocampal subregion and liver from 3 month and 20 month old animals treated orally with vehicle or SAHA and mRNA-Seq of CA1 of 10 month old WT or APP/PS1-21 transgenic animals treated orally with vehicle or SAHA\nAging and increased amyloid burden are major risk factors for cognitive diseases such as Alzheimer's Disease (AD). An effective therapy does not yet exist. Here we use mouse models for age-associated memory impairment and amyloid deposition to study transcriptome and cell type-specific epigenome plasticity at the systems level in the brain and in peripheral organs. We show that at the level of epigenetic gene-expression aging and amyloid pathology are associated with inflammation and impaired synaptic function in the hippocampal CA1 region. While inflammation is associated with increased gene-expression that is linked to a subset of transcription factors, de-regulation of plasticity genes is mediated via different mechanisms in the amyloid and the aging model. Amyloid pathology impairs histone-acetylation and decreases expression of plasticity genes while aging affects differential splicing that is linked to altered H4K12 acetylation at the intron-exon junction in neurons but not in non-neuronal cells. We furthermore show that oral administration of the clinically approved histone deacetylase inhibitor Vorinostat not only restores spatial memory, but also exhibits an anti-inflammatory action and reinstates epigenetic balance and transcriptional homeostasis at the level of gene expression and exon usage.  This is the first systems-level investigation of transcriptome plasticity in the hippocampal CA1 region in aging and AD models and of the effects of an orally dosed histone deacetylase inhibitor. Our data has important implications for the development of minimally invasive and cost-effective therapeutic strategies against age-associated cognitive decline. In fact, our data strongly suggest to test Vorinostat in patients suffering from AD.\nMus musculus\nExpression profiling by high throughput sequencing\nmRNA profile from aged (CA1 and liver) and APP/PS1 (CA1) animals treated with oral vehicle or SAHA for 4 weeks\n",
 "Genome-wide analysis of H4K12 acetylation in neuronal and non-neuronal nuclei in young animals treated with vehicle, WT littermates of APP/PS1-21 animals treated with vehicle and aged and APP/PS1-21 animals treated with oral vehicle or SAHA for 4 weeks\nAging and increased amyloid burden are major risk factors for cognitive diseases such as Alzheimer's Disease (AD). An effective therapy does not yet exist. Here we use mouse models for age-associated memory impairment and amyloid deposition to study transcriptome and cell type-specific epigenome plasticity at the systems level in the brain and in peripheral organs. We show that at the level of epigenetic gene-expression aging and amyloid pathology are associated with inflammation and impaired synaptic function in the hippocampal CA1 region. While inflammation is associated with increased gene-expression that is linked to a subset of transcription factors, de-regulation of plasticity genes is mediated via different mechanisms in the amyloid and the aging model. Amyloid pathology impairs histone-acetylation and decreases expression of plasticity genes while aging affects differential splicing that is linked to altered H4K12 acetylation at the intron-exon junction in neurons but not in non-neuronal cells. We furthermore show that oral administration of the clinically approved histone deacetylase inhibitor Vorinostat not only restores spatial memory, but also exhibits an anti-inflammatory action and reinstates epigenetic balance and transcriptional homeostasis at the level of gene expression and exon usage.  This is the first systems-level investigation of transcriptome plasticity in the hippocampal CA1 region in aging and AD models and of the effects of an orally dosed histone deacetylase inhibitor. Our data has important implications for the development of minimally invasive and cost-effective therapeutic strategies against age-associated cognitive decline. In fact, our data strongly suggest to test Vorinostat in patients suffering from AD.\nMus musculus\nGenome binding/occupancy profiling by high throughput sequencing\nExamination of H4K12 acetylation in neuronal and non-neuronal nuclei of young/old animals and WT/APP/PS1-21 animals. Inputs are also provided.\n",
 'A restricted period for formation of Outer Subventricular Zone defined by Cdh1 and Trnp1 levels\nThe Outer Subventricular Zone (OSVZ) is a germinal layer playing key roles in the development of the neocortex, with particular relevance in gyrencephalic species like human and ferret where it contains abundant basal Radial Glia Cells (bRGCs) that promote cortical expansion.  Here we identify a brief period in ferret embryonic development when apical RGCs generate a burst of bRGCs that become founders of the OSVZ.  After this period, bRGCs in the OSVZ proliferate and self-renew exclusively locally, thereby forming a self-sustained lineage independent from the other germinal layers.  The time window for the brief period of OSVZ bRGC production is delineated by the coincident down-regulation of Cdh1 and Trnp1, and their up-regulation reduces bRGC production and prevents OSVZ seeding.  This mechanism in cortical development may have key relevance in brain evolution and disease\nMustela putorius furo\nExpression profiling by array\nSamples were analyzed with 3 replicates of each of them (except E34SVZ that has 2 replicantes). Comparisons were done taking different reference sample depending on the comparison.\n',
 'Sharp changes in gene expression levels along germinal layers distinguish the development of gyrencephaly\nGyrencephalic species develop folds in the cerebral cortex in a stereotypic manner, but the genetic mechanisms underlying this unique patterning process are unknown. We present a large-scale transcriptomic analysis of individual germinal layers in the developing cortex of the gyrencephalic ferret, comparing between regions prospective of fold and fissure. We find unique transcriptional signatures in each germinal compartment, indicating that thousands of genes are differentially expressed between regions, including ~80% of genes mutated in human cortical malformations. Regional differences result from sharp changes in gene expression levels, which occur at multiple locations across the developing cortex of ferret and human, but not in the lissencephalic mouse. These complex expression patterns emerge sequentially during development and map the eventual location of specific folds and fissures. Our findings demonstrate that step-wise gene expression maps within germinal layers distinguish the development of gyrencephaly from lissencephaly, and may contribute to identify novel genes responsible for human malformations.\nMustela putorius furo\nExpression profiling by array\nSix samples were analyzed with 3 replicates of each of them. Comparisons were done taking different reference sample depending on the comparison ( Usually taking the SG as reference)\n',
 'Natural Variation in Fish Transcriptomes: Comparative Analysis of the Fathead Minnow (Pimephales promelas) and Zebrafish (Danio rerio)\nThis SuperSeries is composed of the SubSeries listed below.\nPimephales promelas\nExpression profiling by arrayThird-party reanalysis\nRefer to individual Series\n',
 'Natural Variation in Fish Transcriptomes: Comparative Analysis of the Zebrafish (Danio rerio)\nFathead minnow and zebrafish are among the most intensively studied fish species in environmental toxicogenomics.  To aid the assessment and interpretation of subtle transcriptomic effects from treatment conditions of interest, there needs to be a better characterization and understanding of the natural variation in gene expression among fish individuals within populations.  Little effort, however, has been made in this area.  Leveraging the transcriptomics data from a number of our toxicogenomics studies conducted over the years, we conducted a meta-analysis of nearly 600 microarrays generated from the ovary tissue of untreated, reproductively mature fathead minnow and zebrafish samples.  As expected, there was considerable batch-to-batch transcriptomic variation; this â\x80\x9cbatch-effectâ\x80\x9d appeared to impact the fish transcriptomes randomly.  The overall level of variation within-batch was quite low in fish ovary tissue, making it a suitable system for studying chemical stressors with subtle biological effects.  The within-batch variation, however, differed considerably among individual genes and molecular pathways.  This difference in variability is probably both technical and biological, thus suggesting a need to take into account both the expression levels and variance in evaluating and interpreting the transcriptional impact on genes and pathways by experimental conditions.  There was significant conservation of both the genomes and transcriptomes between fathead minnow and zebrafish.  The conservation to such a degree would enable not only a comparative biology approach in studying the mechanisms of action underlying environmental stressors, but also effective sharing of a large amount of existing public transcriptomics data for future development of toxicogenomics applications.\nThird-party reanalysisExpression profiling by array\nRNA from treated and control samples were directly hybridized on individual arrays.This study is a reanalysis of ovary Samples from GSE38070. The data processing is different and consisted of:Microarray image processing was performed using the latest Agilent\'s Feature Extraction software available at the time following the manufacturer?s protocol and recommendations.  Data processing was conducted using R package Limma in the order of backgroundCorrect("normexp"), normalizeWithinArrays("loess"), and normalizeBetweenArrays("quantile").  Various types of control probes were filtered out.  Each expression value represents the average of its duplicated probes. Only intensity data from control channel were utilized in the study after channel splitting.The supplementary files attached contain log2 ratios(M) and log2 intensities (A) after backgroundCorrect("normexp"), normalizeWithinArrays("loess"), and normalizeBetweenArrays("quantile") in Limma; various types of control probes were filtered\n',
 'Natural Variation in Fish Transcriptomes: Comparative Analysis of the Fathead Minnow (Pimephales promelas)\nFathead minnow and zebrafish are among the most intensively studied fish species in environmental toxicogenomics.  To aid the assessment and interpretation of subtle transcriptomic effects from treatment conditions of interest, there needs to be a better characterization and understanding of the natural variation in gene expression among fish individuals within populations.  Little effort, however, has been made in this area.  Leveraging the transcriptomics data from a number of our toxicogenomics studies conducted over the years, we conducted a meta-analysis of nearly 600 microarrays generated from the ovary tissue of untreated, reproductively mature fathead minnow and zebrafish samples.  As expected, there was considerable batch-to-batch transcriptomic variation; this â\x80\x9cbatch-effectâ\x80\x9d appeared to impact the fish transcriptomes randomly.  The overall level of variation within-batch was quite low in fish ovary tissue, making it a suitable system for studying chemical stressors with subtle biological effects.  The within-batch variation, however, differed considerably among individual genes and molecular pathways.  This difference in variability is probably both technical and biological, thus suggesting a need to take into account both the expression levels and variance in evaluating and interpreting the transcriptional impact on genes and pathways by experimental conditions.  There was significant conservation of both the genomes and transcriptomes between fathead minnow and zebrafish.  The conservation to such a degree would enable not only a comparative biology approach in studying the mechanisms of action underlying environmental stressors, but also effective sharing of a large amount of existing public transcriptomics data for future development of toxicogenomics applications.\nPimephales promelas\nExpression profiling by array\ntotal RNA from the ovary tissue of treated or control fish labeled in single color was hybridized to Agilent fathead minnow microarray (design 019597)\n',
 'DNA methylation variation in Arabidopsis has a genetic basis and appears to be involved in local adaptation\nWe investigated DNA methylation variation in Swedish Arabidopsis thaliana accessions. We found that methylation of transposable elements is temperature sensitive and associated with genetic polymorphism in both cis and trans, whereas gene body methylation is associated with genetic polymorphism in trans. Additionally, complementary RNA-Seq data for the Arabidopsis accessions were used to correlate methylation changes with gene expression across environments.\nArabidopsis thaliana\nExpression profiling by high throughput sequencing\nmRNA-sequencing (mRNA-Seq) of 160 Arabidopsis thaliana accessions grown at 10 C and 163 grown at 16 C. The source tissue for RNA collection was whole rosette at the 9-leaf stage.\n',
 'Effects of Changes in Food Supply at the Time of Sex Differentiation on the Gonadal Transcriptome of Juvenile Fish. Implications for Natural and Farmed Populations\nBackground: Food supply is a major factor influencing growth rates in animals. This has important implications for both natural and farmed fish populations, since food restriction may difficult reproduction. However, a study on the effects of food supply on the development of juvenile gonads has never been transcriptionally described in fish.Methods and Findings: This study investigated the consequences of growth on gonadal transcriptome of European sea bass in: 1) 4-month-old sexually undifferentiated fish, comparing the gonads of fish with the highest vs. the lowest growth, to explore a possible link between transcriptome and future sex, and 2) testis from 11-month-old juveniles where growth had been manipulated through changes in food supply. The four groups used were: i) sustained fast growth, ii) sustained slow growth, iii) accelerated growth, iv) decelerated growth. The transcriptome of undifferentiated gonads was not drastically affected by initial natural differences in growth. Further, changes in the expression of genes associated with protein turnover were seen, favoring catabolism in slow-growing fish and anabolism in fast-growing fish. Moreover, while fast-growing fish took energy from glucose, as deduced from the pathways affected and the analysis of protein-protein interactions examined, in slow-growing fish lipid metabolism and gluconeogenesis was favored. Interestingly, the highest transcriptomic differences were found when forcing initially fast-growing fish to decelerate their growth, while accelerating growth of initially slow-growing fish resulted in full transcriptomic convergence with sustained fast-growing fish.Conclusions: Food availability during sex differentiation shapes the juvenile testis transcriptome, as evidenced by adaptations to different energy balances. Remarkably, this occurs in absence of major histological changes in the testis.Thus, fish are able to recover transcriptionally their testes if they are provided with enough food supply during sex differentiation; however, an initial fast growth does not represent any advantage in terms of transcriptional fitness if later food becomes scarce.\nDicentrarchus labrax\nExpression profiling by array\nWhole gonad transcriptome analysis from 35 individuals. Two sampling points at 133 days post fertilization (dpf) (7 individuals from the Fast growing group F and 8 from the Slow growing one S and at 332 dpf (5 individuals from each of the following groups: Fast growing-Fast growing FF; Fast growing-Slow growing FS; Slow growing-Fast growing SF and Slow growing-Slow growing SS).\n',
 'DNA methylation variation in Arabidopsis has a genetic basis and appears to be involved in local adaptation\nHere we investigate DNA methylation variation in Swedish Arabidopsis thaliana accessions, demonstrating that methylation of transposable elements is temperature sensitive and associated with genetic polymorphism in both cis and trans, whereas gene body methylation is highly correlated with climate of origin and associated with genetic polymorphism in trans that shows evidence of local adaptation. While genome-wide surveys of naturally occurring DNA methylation have been published previously, the degree of genetic control revealed here is unprecedented. Furthermore, the observation that DNA methylation is associated with climate, and is apparently adaptively important, is completely novel.\nArabidopsis thaliana\nMethylation profiling by high throughput sequencing\nBisulfite sequencing of 152 Swedish Arabidobsis accessions grown at 10 C and 121 grown at 16 C\n',
 'Perinatal-Estrogen-Induced Changes in Gene Expression Related to Brain Sexual Differentiation in Mice\nSexual dimorphism of the behaviors or physiological functions in mammals is mainly due to the sex difference of the brain.  The goal of this study is to identify genes mediating sexaul dimorphism of the brain.The large-scale analysis with microarray in the present study is an attempt to obtain the candidate gene(s) mediating the perinatal estrogen effect causing the brain sexual differentiation.\nMus musculus\nExpression profiling by array\nThirty female mice were injected with estradiol benzoate (EB) or vehicle on the day of birth, and the hypothalamus was collected at either 1, 3, 6, 12, or 24 h after the EB injection.\n',
 'Genome-wide DNA hydroxymethylation changes are associated with neurodevelopmental genes in the developing human brain\n5-hydroxymethylcytosine (5-hmC) is a  newly discovered  modified form of cytosine that has been suspected to  be an important epigenetic modification  in  neurodevelopment. While  DNA  methylation dynamics  have already been implicated during  neurodevelopment, little is known about hydroxymethylation in this process. Here we report DNA hydroxymethylation dynamics during  cerebellum development in  the  human brain.  Overall, we find a positive correlation  between 5-hmC levels and cerebellum development. Genome-wide profiling reveals that 5-hmC is highly enriched  on specific gene regions, including  exons and especially the  untranslated regions (UTRs), but it is depleted on introns and intergenic regions. Furthermore, we have identified fetus-specific and adult-specific differentially hydroxymethylated regions (DhMRs), most of which overlap with genes and  CpG island shores. Surprisingly, during development DhMRs are highly enriched in genes encoding mRNAs that can be regulated by fragile X mental retardation protein (FMRP), some of which are disrupted in autism, as well as in many known autism genes. Our results suggest that 5-hmC-mediated epigenetic regulation  may  broadly impact the development of  the  human brain, and  its dysregulation could  contribute to the molecular pathogenesis of neurodevelopmental disorders.\nHomo sapiens\nMethylation profiling by high throughput sequencing\nWe generated comprehensive genome-wide profiles of 5hmC in human cerebellum.\n',
 'Expression and DNA methylation of colon cancer cell lines\nThis SuperSeries is composed of the SubSeries listed below.\nHomo sapiens\nExpression profiling by arrayMethylation profiling by genome tiling array\nRefer to individual Series\n',
 'DNA methylation analysis of Colon Cancer cell line SW48\nWe combined gene expression experiments in response to HDAC inhibitor Depsipeptide with DNA methylation level to assess gene reactivation arising from hypermethylated promoters.\nHomo sapiens\nMethylation profiling by genome tiling array\nWe did one experiment with 1 untreated and 1 treated batch of cells\n',
 'Colon Cancer cell line YB5 untreated cells vs YB5 treated 24h with Depsipeptide at 20 nM\nStudy the effect of HDACi Depsipeptide on gene expression. We combined these data with whole genome DNA methylation microarray to test the reactivation of genes that have an hypermethylated promoter region.\nHomo sapiens\nExpression profiling by array\nWe did two independant experiments with 2 untreated and 2 treated different batch of cells\n',
 'Genome-wide survey of large rare copy number variations in Alzheimerâ\x80\x99s disease   among Caribbean Hispanics\nRecently genome-wide association studies have identified significant association between Alzheimerâ\x80\x99s disease and variations in CLU, PICALM, BIN1, CR1, MS4A4/MS4A6E, CD2AP, CD33, EPHA1 and ABCA7. However, the pathogenic variants in these loci have not yet been found. We conducted a genome-wide scan for large copy number variations (CNVs) in a dataset of Caribbean Hispanic origin (554 controls and 559 cases with late-onset Alzheimerâ\x80\x99s disease) that was previously investigated in a SNP-based genome-wide association study using Illumina HumanHap 650Y platform. We ran four CNV calling algorithms and analyzed rare large CNVs (>100 Kb) to obtain high-confidence calls that were detected by at least two algorithms. In total, 734 such CNVs were observed in our dataset. Global burden analyses did not reveal significant differences between cases and controls in CNV rate, distribution of deletions or duplications, total or average CNV size; and number of genes affected by CNVs. However, we observed a nominal association between Alzheimerâ\x80\x99s disease and a ~470 Kb duplication on chromosome15q11.2 (P=0.037). This duplication, encompassing up to five genes (TUBGCP5, CYFIP1, NIPA2, NIPA1 and WHAMML1) was present in 10 cases (2.6%) and 3 controls (0.8%). The dosage increase of CYFIP1 and NIPA1 genes was further confirmed by quantitative PCR. The current study did not detect CNVs (including common CNVs) that affect novel Alzheimerâ\x80\x99s disease loci reported by large genome-wide association studies. However, since the array technology used in our study has limitations in detecting small CNVs, future studies must carefully assess novel AD associated genes for the presence of disease related CNVs.\nHomo sapiens\nGenome variation profiling by SNP array\nCase-control analysis, screening of large copy number variation in 559 Alzheimer cases and 554 control subjects of Caribbean Hispanic ancestry\n',
 'A DNA Methylation Fingerprint of  1,628 Human Samples\nDNA methylation is the best characterized of the different layers that make up the epigenetic setting. Most of the studies characterizing DNA methylation patterns have been restricted to particular genomic loci in a limited number of human samples and pathological conditions. The recently arrived single-base-resolution technologies for DNA methylation are extremely helpful tools, but are not yet applicable and affordable for studying large groups of subjects. Herein, we present a compromise between an extremely comprehensive study of a human sample population with an intermediate level of resolution of CpGs at the genomic level. We obtained a DNA methylation fingerprint of 1,628 human samples where we interrogated 1,505 CpG sites. The DNA methylation patterns revealed show this epigenetic mark to be critical in tissue-type definition and stemness, particularly around transcription start sites that are not within a CpG island. For disease, the generated DNA methylation fingerprints show that, during tumorigenesis, human cancer cells underwent a progressive gain of promoter CpG island hypermethylation and a loss of CpG methylation in non-CpG island promoters. Although transformed cells are those where DNA methylation disruption is more obvious, we observed that other common human diseases, such as neurological and autoimmune disorders, had their own distinct DNA methylation profiles. Most importantly, we provide proof of principle that the obtained DNA methylation fingerprints might be useful for translational purposes by showing that are able to identify the tumor type origin of Cancers of Unknown Primary (CUPs). Thus, the DNA methylation patterns identified across the largest spectrum of samples, tissues and diseases reported to date constitute a baseline for developing higher-resolution DNA methylation maps, and provide important clues concerning the contribution of CpG methylation to tissue identity and its changes in the most prevalent human diseases.\nHomo sapiens\nMethylation profiling by array\nIt was studied the genomic DNA from 1,628 human samples corresponding to 424 normal tissues (180 leukocytes, 97 colon mucosa and 227 other normal samples), 1,054 tumorigenic samples (premalignant lesions, primary tumors and metastases) and 150 non-cancerous disorders, such as brain lesions from Alzheimerâ\x80\x99s disease, dementia with Lewy bodies, aortic atherosclerotic lesions, myopathies and autoimmune disorders.\n',
 'Analysis and design of RNA sequencing experiments for identifying mRNA isoform regulation\nThrough alternative splicing, most human genes express multiple isoforms that may have distinct or even antagonistic functions. To infer isoform regulation based on data from high-throughput sequencing of cDNA fragments (RNA-Seq), we have developed MISO, a computational model that estimates the expression level of alternatively spliced exons and mRNA isoforms and provides intuitive measures of conï¬\x81dence in these estimates. Incorporation of the length distribution of inserted cDNA fragments in paired-end RNA-Seq analysis in MISO enables dramatic improvements in estimation of alternative splicing levels relative to previous methods. We show that one lane of paired-end RNA-Seq data can provide far more information about splicing than two lanes of single-end data, depending critically on properties of the distribution of cDNA fragment lengths in the sequenced library. MISO also leads to an intuitive method to detect diï¬\x80erentially regulated exons  or isoforms. Application of this method implicates the RNA splicing factor hnRNP H in regulation of alternative cleavage and polyadenylation, a role that is supported by UV  crosslinking/immunoprecipitation/high-throughput sequencing (CLIP-Seq) analysis. Together, our results provide a probabilistic framework for RNA-Seq analysis, derive functional insights into pre-mRNA processing, and yield guidelines for the optimal design of RNA-Seq experiments for studies of gene and isoform expression.\nExpression profiling by high throughput sequencingOther\nCLIPseq of hnRNP H in HEK 293T cells.RNAseq of polyA+ RNA from C2C12 mouse myoblasts stably expressing an empty vector or a vector containing an shRNA against CUGBP1. Libraries of two different insert lengths were created and examined.\n',
 'Human LSD2/KDM1b/AOF1 regulates gene transcription by modulating H3K4me2 methylation\nThis SuperSeries is composed of the SubSeries listed below.\nHomo sapiens\nGenome binding/occupancy profiling by genome tiling arrayExpression profiling by array\nRefer to individual Series\n',
 'Human LSD2/KDM1b/AOF1 regulates gene transcription by modulating intragenic H3K4me2 methylation (ChIP-chip)\nDynamic histone H3K4 methylation is an important epigenetic component of transcriptional regulation. However, most of our current understanding of this histone mark is confined to regulation of transcriptional initiation. We now show that human LSD2/KDM1b/AOF1, the human homolog of LSD1, is a novel H3K4me1/2 demethylase that specifically regulates histone H3K4 methylation within intragenic regions of its target genes. Genome-wide mapping reveals that LSD2 associates predominantly with the gene bodies of actively transcribed genes, but is markedly absent from promoters. Depletion of endogenous LSD2 results in an increase of H3K4me2 as well as a decrease of H3K9me2 at LSD2 binding sites, and a consequent dysregulation of target gene transcription. Furthermore, characterization of LSD2 complex revealed that LSD2 forms active complexes with euchromatic histone methyltransferases EHMT1/2 and NSD3 as well as cellular factors involved in active transcription elongation. These data provide a possible molecular mechanism linking LSD2 to transcriptional regulation post initiation.\nHomo sapiens\nGenome binding/occupancy profiling by genome tiling array\nNative chromatin of HeLa-S stably expressing FgHA-LSD2 was prepared by micrococcal nuclease digestion as previously described (Umlauf et al., 2004), and was primarily composed of mono-, di-, and tri-nucleosomes. Native chromatin fragments were sequential immunoprecipited using anti-flag and anti-HA agarose beads (Sigma). After extensive washing, Flag or HA peptides (Sigma) were used for elution after each immunoprecipitation. DNA from the final step was purified and processed for tiling array hybridization to Human Genome tiling 2.0 Array (Affymetrix) per manufactureâ\x80\x99s instructions. Input DNA was used as control. Regions enriched by TAP N-ChIP were identified using algorithm Model-Based Analysis of Tiling arrays (MAT) (Johnson et al., 2006), and displayed by Integrated Genome Browser (IGB). CEAS (cis-element annotation system) was used for statistic analysis of the distribution pattern of LSD2 binding sites on promoter and non-promoter regions (Ji et al., 2006). To estimate the statistical significance of colocalization between LSD2 and histone modification marks previously identified (Barski et al., 2007), we estimated the null distribution of overlap by generating 100 randomly permutated sample of the histone modification loci and the overlap is calculated for each permutated sample.\n',
 'Human LSD2/KDM1b/AOF1 regulates gene transcription by modulating H3K4me2 methylation (Affymetrix)\nDynamic histone H3K4 methylation is an important epigenetic component of transcriptional regulation. However, most of our current understanding of this histone mark is confined to regulation of transcriptional initiation. We now show that human LSD2/KDM1b/AOF1, the human homolog of LSD1, is a novel H3K4me1/2 demethylase that specifically regulates histone H3K4 methylation within intragenic regions of its target genes. Genome-wide mapping reveals that LSD2 associates predominantly with the gene bodies of actively transcribed genes, but is markedly absent from promoters. Depletion of endogenous LSD2 results in an increase of H3K4me2 as well as a decrease of H3K9me2 at LSD2 binding sites, and a consequent dysregulation of target gene transcription. Furthermore, characterization of LSD2 complex revealed that LSD2 forms active complexes with euchromatic histone methyltransferases EHMT1/2 and NSD3 as well as cellular factors involved in active transcription elongation. These data provide a possible molecular mechanism linking LSD2 to transcriptional regulation post initiation.We used microarray analysis to identify the subset of genes that are differentially expression after depletion of endogenous LSD2/KDM1b/AOF1.\nHomo sapiens\nExpression profiling by array\nRetroviral shRNA targeting human LSD2 (5â\x80\x99-GTGGGACCACAATGAATTCTT -3â\x80\x99) and control shRNA was used to infect HeLa. RNA was purified 70 hours post infection and processed for Human Genome U133 Plus 2.0 Array (Affymetrix) hybridization per manufactureâ\x80\x99s instructions. Biological duplicates were analyzed.\n',
 'A tripartite paternally methylated region within the Gpr1-Zdbf2 imprinted domain on mouse chromosome 1 identified by meDIP-on-chip\nThis SuperSeries is composed of the SubSeries listed below.\nMethylation profiling by genome tiling array\nRefer to individual Series\n',
 'Methylation status in human sperm\nGenomic imprinting describes the expression of a subset of mammalian genes from one parental chromosome. The parent-of-origin specific expression of imprinted genes relies on DNA methylation of CpG-dinucleotides at differentially methylated regions (DMRs) during gametogenesis. We identified the paternally methylated DMR at human chromosome 2 near the imprinted ZDBF2 gene using a methylated-DNA immunoprecipitation-on-chip (meDIP-on-chip) method applied to DNA from sperm.\nHomo sapiens\nMethylation profiling by genome tiling array\nTo analyze whether or not the GPR1-ZDBF2 DMR is conserved in human genome, methylation analysis of human sperm sample was performed using MeDIP and genome tiling array.\n',
 'Methylation status in AG- and PG-derived cells and sperm\nGenomic imprinting describes the expression of a subset of mammalian genes from one parental chromosome. The parent-of-origin specific expression of imprinted genes relies on DNA methylation of CpG-dinucleotides at differentially methylated regions (DMRs) during gametogenesis. We identified the paternally methylated DMRs at mouse chromosome 1 near the imprinted Zdbf2 gene using a methylated-DNA immunoprecipitation-on-chip (meDIP-on-chip) method applied to DNA from parthenogenetic (PG)- and androgenetic (AG)-derived cells and sperm.\nMus musculus\nMethylation profiling by genome tiling array\nTo identify novel DMRs, genome-wide methylation analysis of three samples were performed using MeDIP and whole genome tiling array.\n',
 'Nucleosome deposition and DNA methylation at coding region boundaries\nWe profiled CpG methylation for human T cells and compared this pattern with public T cell nucleosome (H2A.Z) and polymerase II profiles (SRA000234). Focusing on DNA regions surrounding the start codon and stop codon, instead of the transcription start and end sites, we discovered a very intriguing pattern, namely methylation and nucleosomal peaks at those regions, more prominent than peaks near transcript boundaries. Elongating polymerases tend to pause near both coding ends immediately upstream of the epigenetic peaks, causing a significant reduction in elongation efficiency, suggesting their role in control of RNA splicing. Conserved features in underlying protein coding sequences seem to dictate their evolutionary conservation across multiple species. The nucleosomal and methylation marks are commonly associated with high sequence-encoded DNA-bending propensity but differentially with CpG density.Keywords: Epigenetics\nHomo sapiens\nMethylation profiling by high throughput sequencing\nSample submission examines CpG methylation from human T cells.\n',
 'Abundant Quantitative Trait Loci for CpG Methylation and Expression Across Human Brain Tissues\nA fundamental challenge in the post-genome era is to understand and annotate the consequences of genetic variation, particularly within the context of human tissues. We describe a set of integrated experiments designed to investigate the effects of common genetic variability on DNA methylation, mRNA expression and microRNA (miRNA) expression in four distinct human brain regions. We show that brain tissues may be readily distinguished based on methylation status or expression profile. We find an abundance of genetic cis regulation mRNA expression and show for the first time abundant quantitative trait loci for DNA CpG methylation. We observe that the largest magnitude effects occur across distinct brain regions. We believe these data, which we have made publicly available, will be useful in understanding the biological effects of genetic variation.Authorized Access data:Mapping of GEO sample accessions to dbGaP subject/sample IDs is available through dbGaP Authorized Access, see http://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs000249.v1.p1\nHomo sapiens\nExpression profiling by arrayNon-coding RNA profiling by arrayMethylation profiling by array\nBecause of our interest in genomic regulation of expression and neurological disorders we embarked upon a series of experiments to provide a brain region-specific contextual framework for genetic and epigenetic regulation of gene expression. We obtained frozen brain tissue from the cerebellum, frontal cortex, pons and temporal cortex from 150 subjects (total 600 tissue samples). We undertook four separate assays across this series; first, genome-wide SNP genotyping; second, assay of >27,000 CpG methylation sites in each of the four brain regions; third, mRNA expression profiling of >22,000 transcripts in all four brain regions; and, fourth, miRNA expression profiling of 735 miRNA transcripts. Here we discuss the results of these experiments, particularly in the context of integrated datasets to define expression and CpG methylation quantitative trait loci (eQTL and methQTL) and detailing differences and similarities across brain regions.\n',
 "Transcriptional profiles of rodent hippocampal CA1 tissue during aging and cognitive decline\nRecent microarray studies in the hippocampus of rodents or Alzheimerâ\x80\x99s disease (AD) subjects have identified a substantial number of cellular pathways/processes correlated with aging and cognitive decline. However, the temporal relationships among these expression changes or with cognitive impairment have not been studied in depth.  Here, using Affymetrix microarrays, immunohistochemistry and Morris water maze cognitive testing across 5 age groups of male F344 rats (n=9-15/group, one microarray per animal), we systematically analyzed the temporal sequence and cellular localization of aging changes in expression.  These were correlated with performance scores on the hippocampus-dependent Morris Water Maze task. Significant microarray results were sorted in to Early, Intermediate, Midlife, and Late patterns of expression, and functionally categorized (Early- downregulated neural development, lipid synthesis and energy-utilization; upregulated ribosomal synthesis, growth, stress/inflammatory, lysosome and protein/lipid degradation. Intermediate- increased defense/inflammatory activation and decreased transporter activity; Midlife- downregulated energy-dependent signaling and neurite growth, upregulated astroglial activation, Ca2+-binding, cholesterol/lipid trafficking, myelinogenic processes and additional lysosome/inflammation; Late- further recruitment of genes in already-altered pathways). Immunohistochemistry revealed a primarily astrocytic localization of the processes upregulated in midlife, as well as increased density of myelin proteins. Evidence of cognitive impairment first appeared in the 12-month-old group (midlife) and was increased further in the 23-month-old group, exhibiting the highest correlations with some upregulated genes related to cholesterol transport (e.g., Apoe, Abca2), protein management and ion binding. Some upregulated genes for inflammation (Il6st) and myelinogenesis (Pmp22) also correlated with impairment. Together, the data are consistent with a novel sequential cascade model of brain aging in which metabolic alterations early in maturity are followed by inflammation and midlife activation of an astrocyte-centered cholesterol trafficking pathway that stimulates oligodendrocyte remyelination programs. Importantly, this cholesterol trafficking pathway also may compete for astroglial bioenergetic support of neurons, in turn, leading to downregulation of energy-dependent pathways needed to sustain cognitive functions.Keywords: aging time course\nRattus norvegicus\nExpression profiling by array\nFisher 344 rats aged 3 (n = 9), 6 (n = 9), 9 (n = 9), 12 (n = 9), or 25 (n = 15; but see below) months were behaviorally characterized on the Morris water maze. Briefly, animals were placed in a circular water bath chamber with a partially submerged platform. Three times per day for three days, animals were placed at randomized starting points with the goal of locating the hidden platform. Generally, animals became better at locating the platform, and by the third day of training, were reliably swimming directly to the platform despite randomized starting loci. On the fourth day, the platform was removed and the degree to which the animals focused their searches on the area that previously contained the platform was measured. On the last day of behavior, animals were given a cue trial in which the platform was raised above the water level so that the rats could see it. Measures taken from the behavior study included path length (how long did the animal swim prior to locating the platform/ platform area), latency to platform in seconds, and velocity (centimeters/ second; swim speed). Two aged animals (E2 and E12) were removed from the study for poor health, reducing the 23 month cohort (n = 13). Twelve to twenty four hours after the last behavioral session, animals were killed and their hippocampi removed. The right hippocampus went to immunohistochemistry for quantification and localization of selected protein products. The left hippocampus was dissected, the CA1 region removed and frozen in dry ice prior to microarray processing. mRNA was extracted from each animal's tissue by standard Affymetrix protocols (see Blalock et al., 2003) and hybridized to individual microarrays (RAE 230A; N = 49 arrays in total). Results were processed with the MAS5 algorithm and subjected to â\x80\x98all probe setâ\x80\x99 scaling with a target intensity of 500. Probe sets were filtered for presence (> = 5 presence calls study wide) and gene symbol level annotation prior to statistical analysis. 1-ANOVA across age was used to identify aging-related probe sets/genes (p <= 0.05). Aging-related genes were subjected to post hoc template correlation to determine the pattern of age-related change, and further, within the 12 and 23 month old groups, Pearson's correlation with behavioral scores was used to identify genes whose expression levels correlated significantly (p <= 0.05) with behavior. Twelve and 23 month old groups were chosen for this correlation analysis as they showed the greatest variability among similarly aged animals, suggesting that these ages show important cognitive transition phases during aging-related cognitive decline. Functional grouping analysis was performed using the DAVID suite of on-line tools.\n",
 '']